In [ ]:
#From https://github.com/gabrielo/Allegheny-County-Property-Assessments

In [ ]:
import json, os, re, sqlite3, thread, urllib, urllib2
def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('timelapse-utilities.ipynb')

In [ ]:
# assessments/cd/AC Property Assessments_10012017.xls (which is actually TDF) from Amy Gottsegen and Randy Sargent buying a CD from the county assessors office
# assessments/Allegheny_County_Parcel_Boundaries.geojson from https://data.wprdc.org/dataset/allegheny-county-parcel-boundaries
import array, csv, datetime, json, math, numpy, os, random, re 
from dateutil.parser import parse
from shapely.geometry import mapping, shape
from PIL import Image
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import pandas,numbers
from operator import itemgetter, attrgetter
import string
import calendar,time
from IPython.core.display import HTML

# Use geopandas.  Boilerplate from https://docs.google.com/document/d/1utZuLHcKQEZNXTQLOysTNCxTHrqxczAUymmtplpn27Q/edit#heading=h.f50xoxwmcir
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
%matplotlib inline

pd.options.display.max_colwidth = 300
pd.options.display.max_rows = 100
pd.set_option('display.max_columns', 500)

# Built-in time handling fails for times before 1900.  Use arrow instead.  
# See https://arrow.readthedocs.io/en/latest/ for info
import arrow

def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

# This does the same as above, but takes a GeoJSON point (which is what centroid returns)
def PointToPixelXY(point):
    lon=point.x
    lat=point.y
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

def GetCentroid(geometry):
    s = shape(geometry)
    return (s.centroid.x, s.centroid.y)   

def GetEpoch(date):
    return (date - datetime.datetime(1970, 1, 1)).total_seconds()

def HexToRgb(hex_string):
    rgb = colors.hex2color(hex_string)
    r,g,b = tuple([int(255*x) for x in rgb])
    return (r,g,b)

In [ ]:
from shapely.geometry import Point
import collections


# Google geocoding API to geocode an address
# Assumes key is in google_api_key.txt
# Use as second level lookup only for those which fail census lookup

re_noalnum = re.compile('^([^\w]+)$')

def geocode_address_google(address):
    if(pandas.isnull(address) or re_noalnum.match(address)):
        #print "Skipping address lookup for %r, no alphanumeric characters" %(address)
        return None

    try:
        geocode_address_google.conn
    except:
        geocode_address_google.conn = sqlite3.connect('geocoding_cache')
        geocode_address_google.cur = geocode_address_google.conn.cursor()
        geocode_address_google.cur.execute(('CREATE TABLE IF NOT EXISTS kvs'
                                     ' (key PRIMARY KEY, value)'
                                     ' WITHOUT ROWID;'))
        geocode_address_google.conn.commit()

    geocode_address_google.cur.execute('SELECT value FROM kvs WHERE key=?', (address,))
    rows = geocode_address_google.cur.fetchall()
    if rows:
        #print "Google: Found %r in cache" % (address)
        return json.loads(rows[0][0])
        
    #print "Google: %r not found in cache, fetching" % (address)

    api_key = open('google-api-key-do-not-commit.txt').read()
    payload = {'address':address, 'key':api_key}
    result = json.load(urllib2.urlopen('https://maps.googleapis.com/maps/api/geocode/json?%s' % urllib.urlencode(payload)))
    
    geocode_address_google.cur.execute(('INSERT OR REPLACE INTO kvs (key, value)'
                                 ' VALUES (?, ?);'),
                                 (address, json.dumps(result)))
    geocode_address_google.conn.commit()
    
    # If the result succeeded and generated a formatted address, cache under that key too
    try:
        if(result['status']=='OK'):
            if(len(result['results'])== 1):
                canonical = result['results'][0]['formatted_address']
                #print "Google: Storing canonical addr of %r for %r" % (canonical, address)
                geocode_address_google.cur.execute(('INSERT OR REPLACE INTO kvs (key, value)'
                                                     ' VALUES (?, ?);'),
                                                     (canonical, json.dumps(result)))
                geocode_address_google.conn.commit()
            else:
                print "Google: Fetch %r seemed to succeed, but results empty or non-unique" % (address)

        else:
            print "Google: Fetch %r failed" % (address)

    except:
        #pass
        raise
    
    return result

def get_canonical_address_google(raw_addr):
    if(pandas.isnull(raw_addr) or re_noalnum.match(raw_addr)):
        #print "Skipping address lookup for %r, no alphanumeric characters" %(raw_addr)
        return None

    try:
        result=geocode_address_google(raw_addr)
        if(result['status']=='OK' and len(result['results'])== 1):
            return(result['results'][0]['formatted_address'])
        else:
            return None
    except:
        return None

def get_canonical_zip_google(addr):
    if(pandas.isnull(addr) or re_noalnum.match(addr)):
        #print "Skipping address lookup for %r, no alphanumeric characters" %(addr)
        return None

    try:
        result=geocode_address_google(addr)
        if(result['status']=='OK' and len(result['results'])== 1):
            addr_comps = result['results'][0]['address_components']
            for i in range(0, len(addr_comps)):
                if('postal_code' in addr_comps[i]['types']):
                    return addr_comps[i]['short_name']
        
        return None
    except:
        return None
    
def get_canonical_coords_google(addr):
    if(pandas.isnull(addr) or re_noalnum.match(addr)):
        #print "Skipping address lookup for %r, no alphanumeric characters" %(addr)
        return None

    try:
        result=geocode_address_google(addr)
        if(result['status']=='OK' and len(result['results'])== 1):
            latlon = result['results'][0]['geometry']['location']
            if(latlon):
                return(Point(latlon['lng'], latlon['lat']))
        return None
    except:
        return None


In [ ]:
# Uses Census geocoding API to geocode an address
# Assumes key is in google_api_key.txt

import sqlite3
import urllib,urllib2
import json
from shapely.geometry import Point
import collections

# Match if the string contains no alphanumerics
re_noalnum = re.compile('^([^\w]+)$')

def geocode_address_census(address):
    if(pandas.isnull(address) or re_noalnum.match(address)):
        #print "Skipping address lookup for %r, no alphanumeric characters" %(address)
        return None

    try:
        geocode_address_census.sqlite
    except:
        geocode_address_census.sqlite = {}
    
    tpid = '%d.%s' % (os.getpid(), thread.get_ident())
    if not tpid in geocode_address_census.sqlite:
        conn = sqlite3.connect('geocoding_cache_census')
        cur = conn.cursor()
        geocode_address_census.sqlite[tpid] = { 'conn': conn, 'cur': cur }

        cur.execute(('CREATE TABLE IF NOT EXISTS kvs'
                     ' (key PRIMARY KEY, value)'
                     ' WITHOUT ROWID;'))
        conn.commit()
    else:
        conn = geocode_address_census.sqlite[tpid]['conn']
        cur = geocode_address_census.sqlite[tpid]['cur']

    cur.execute('SELECT value FROM kvs WHERE key=?', (address,))
    rows = cur.fetchall()
    if rows:
        return json.loads(rows[0][0])
    
    payload = {'address':address, 'benchmark':4, 'format':'json'}
    result = json.load(urllib2.urlopen('https://geocoding.geo.census.gov/geocoder/locations/onelineaddress?%s' % urllib.urlencode(payload)))
    
    cur.execute(('INSERT OR REPLACE INTO kvs (key, value)'
                 ' VALUES (?, ?);'),
                (address, json.dumps(result)))
    conn.commit()
    
    if len(result['result']['addressMatches']) == 1:
        canonical = result['result']['addressMatches'][0]['matchedAddress']
        cur.execute(('INSERT OR REPLACE INTO kvs (key, value)'
                     ' VALUES (?, ?);'),
                     (canonical, json.dumps(result)))
        conn.commit()
    
    return result

def cache_canonical_addresses(addrs):
    t = SimpleThreadPoolExecutor(25)
    for addr in addrs:
        if(not (pandas.isnull(addr) or re_noalnum.match(addr))):
            t.submit(geocode_address_census, addr)
        else:
            #print "Skipping address lookup for %r, no alphanumeric characters" %(addr)
            pass
    t.shutdown()

def get_canonical_address_census(raw_addr):
    if(pandas.isnull(raw_addr) or re_noalnum.match(raw_addr)):
        #print "Skipping address lookup for %r, no alphanumeric characters" %(raw_addr)
        return None

    try:
        result=geocode_address_census(raw_addr)
        if len(result['result']['addressMatches']) == 1:
            return(result['result']['addressMatches'][0]['matchedAddress'])
        else:
            return None
    except:
        return None

def get_canonical_zip_census(addr):
    try:
        result=geocode_address_census(addr)
        if len(result['result']['addressMatches']) == 1:
            return(result['result']['addressMatches'][0]['addressComponents']['zip'])
        else:
            return None
    except:
        return None

    
def get_canonical_coords_census(addr):
    try:
        result=geocode_address_census(addr)
        if len(result['result']['addressMatches']) == 1:
            coords = result['result']['addressMatches'][0]['coordinates']
            return(Point(coords['x'], coords['y']))
        else:
            return None
    except:
        return None

################################################################################################################
# These can handle being passed a map with either census or google in it and dispatch to the right service

# TODO: Think if this is the right behavior: If handed a regular string do census lookup
# If handed a map, return census if non-null, then google if non-null
def get_canonical_address(addr):
    if(pandas.isnull(addr) or ((not isinstance(addr, collections.Mapping) and re_noalnum.match(addr)))):
        #print "Skipping address lookup for %r, no alphanumeric characters" %(addr)
        return None

    if(isinstance(addr, collections.Mapping)):
        if('census' in addr and not pd.isnull(addr['census'])):
            return addr['census']
        elif('google' in addr and not pd.isnull(addr['google'])):
            return addr['google']
    else:
        return(get_canonical_address_census(addr))

def get_canonical_zip(addr):
    if(pandas.isnull(addr) or ((not isinstance(addr, collections.Mapping) and re_noalnum.match(addr)))):
        #print "Skipping address lookup for %r, no alphanumeric characters" %(addr)
        return None
    if(isinstance(addr, collections.Mapping)):
        if('census' in addr and not pd.isnull(addr['census'])):
            return get_canonical_zip_census(addr['census'])
        elif('google' in addr and not pd.isnull(addr['google'])):
            return get_canonical_zip_google(addr['google'])
    else:
        return(get_canonical_zip_census(addr))

def get_canonical_coords(addr):
    if(pandas.isnull(addr) or ((not isinstance(addr, collections.Mapping) and re_noalnum.match(addr)))):
        #print "Skipping address lookup for %r, no alphanumeric characters" %(addr)
        return None
    
    if(isinstance(addr, collections.Mapping)):
        if('census' in addr and not pd.isnull(addr['census'])):
            return get_canonical_coords_census(addr['census'])
        elif('google' in addr and not pd.isnull(addr['google'])):
            return get_canonical_coords_google(addr['google'])
    else:
        return(get_canonical_coords_census(addr))

def canonicalize_addr_colset(df, col_arr, out_colname):
    for  col_name in col_arr:
        df[col_name]=df[col_name].astype(basestring)
        df[col_name].fillna('', inplace=True)

    df[out_colname] = df[col_arr].apply(lambda x: get_canonical_address_census(' '.join(x)), axis=1)
    
    

In [ ]:
get_canonical_zip({'google':'4625 CORDAY WAY PITTSBURGH PA 15224'})

In [ ]:
get_canonical_zip('4625 CORDAY WAY PITTSBURGH PA 15224')

# Create structures to populate with sale and ownership information

In [ ]:
# Read in 2017 data to harvest list of residential parcel IDs
# We need to do this before loading the older ADB files because they're missing some of the columns we need.

In [ ]:
# For each residential PARID, we want to generate a sorted list of who bought it when.
# For each owner name, we want to generate a sorted list of PARIDs they bought and when they bought it.
# For each change address, we want to generate a sorted list of PARIDs they bought and when they bought it.
# Each of the following is a map indexed on PARID, owner name, or change address.  

# The map indexed on PARID builds up a history of a given property.
#   Each record contains a a list of maps containing a list of date, event_type, owner name, and change address
#   event_type is either PURCHASE or FORECLOSURE

# Each map indexed on owner name or change address builds a history of a given owner/change address.
#   Each record contains a list of maps containing date, event_type, and PARID
#   event_type is either PURCHASE or SALE


# To start with, clear the maps
property_map = {}
owner_map = {}
changeaddr_map = {}

# Also create a map to cache lookups for owner and changeaddress PARID sets at various times
# to speed up runtime
owned_parids_cache={}

# TODO: Create a map of owner and changeaddress to inferred owner type
# 'GOVERNMENT'
# 'INVESTOR' 

# Create a global map of PARIDs to canonical property addresses.  Not all versions of the db have valid 
# property addresses, but at least one hopefully should!
canonical_property_address_map={}

# Create a global map of PARIDs to raw property addresses.  We might have conflicts, but at least we'll have something
# to fall back on if canonical resolution fails
raw_property_address_map={}

In [ ]:
# Create functions to update the above maps with info from a given row of an assessment spreadsheet/database.  
# The first argument should be a pandas table, the second an index into the table.
# We expect the index to be PARID
# We expect the following columns to exist and be valid:
#   PROPERTYOWNER
#   CHANGENOTICEADDRESS1 - CHANGENOTICEADDRESS4
#   SALEDATE


# In 2017 the date format is MM-DD-YYYY
# In 2009, an example is 09/16/96 00:00:00

# Setup regular expression for parsing various sorts of dates
re_MDY = re.compile('(\d\d)-(\d\d)-(\d\d\d\d)')
re_YMD = re.compile('(\d\d\d\d)-(\d\d)-(\d\d)')
re_MDY_HMS = re.compile('(\d+)/(\d+)/(\d+) (\d+):(\d+):(\d+)')

# Setup this year to use for dealing with 2-digit dates
this_year = datetime.datetime.now().year
this_date = arrow.now().format('YYYY-MM-DD')

# Utility function to normalize SALEDATE format
# SALEDATE field is in '%m-%d-%Y' format (ex 10-26-2012) when present
def SaledateToEpoch(datestr):
    if(re_YMD.match(datestr)!=None):
        return calendar.timegm(time.strptime(datestr, '%Y-%m-%d'))
    elif(re_MDY.match(datestr)!=None):
        return calendar.timegm(time.strptime(datestr, '%m-%d-%Y'))

# Built-in time handling fails for times before 1900.  Use arrow instead.  
# See https://arrow.readthedocs.io/en/latest/ for info
def SaledateToYMD(datestr):
    # Check which format and parse accordingly
    if(re_YMD.match(datestr)!=None):
        # Already like we want it
        return datestr
    if(re_MDY.match(datestr)!=None):
        return arrow.get(datestr, 'MM-DD-YYYY').format('YYYY-MM-DD')
    elif(re_MDY_HMS.match(datestr)!=None):
        # Some of the 2-digit years are farther back than python's default pivot year of 1969
        # If the year we parse is > this_year, set it back by 100 years
        ad=arrow.get(datestr, 'MM/DD/YY HH:mm:ss')
        if(ad.year>this_year):
            ad = ad.replace(year=(ad.year-100))
        return ad.format('YYYY-MM-DD')
    else:
        raise Exception('Unrecognized saledate format %r' % (datestr))

def merge_dwelling_col(df):
    r_df = pd.merge(df,dwelling_col.to_frame(),on='PARID', left_on=None, right_on=None,
                    left_index=False, right_index=False, sort=False,
                    suffixes=('_x', '_y'), copy=True, indicator=False,
                    validate=None)
    print "After merge, size = %d" % (len(r_df))

    r_df = r_df[r_df.is_dwelling]
    print "After filtering to only include dwellings, size = %d" % (len(r_df))
    return(r_df)

# find_event_by_date takes a list of maps containg 'date' fields and looks for one the the last one that's before
# search_date or the first which is simultaneous (in case of multiple simultaneous records).  
# Returns -1 if the search_date is earlier than any existing item.  It assumes that each map in the event list 
# contains a 'date' field and that they're in sorted order from earliest to latest.
def find_event_index_by_date(event_list, search_date):
    match_index=-1
    for i in range(0,len(event_list)):
        if(event_list[i]['date']<search_date):
            # This happened before the date we're looking for, update match_index
            match_index=i
        elif(event_list[i]['date']==search_date):
            # This is the first exact match, return i
            return(i)
        else:
            # This happened after the date we're looking for, return match_index
            return match_index
    # If we get to here, there is no event after this date.  Return match_index
    return match_index

def find_event_index_by_date_and_asofdate(event_list, search_date, asofdate):
    match_index=-1
    for i in range(0,len(event_list)):
        if(event_list[i]['date']<search_date):
            # This happened before the date we're looking for, update match_index
            match_index=i
        elif(event_list[i]['date']==search_date):
            if(event_list[i]['asofdate']==asofdate):
                # This is an exact match, return i
                return(i)
            elif(event_list[i]['asofdate']<asofdate):
                # This happened on the same search_date and before the asoifdate 
                # we're looking for, update match_index
                match_index=i

        else:
            # This happened after the date we're looking for, return match_index
            return match_index
    # If we get to here, there is no event after this date.  Return match_index
    return match_index

# This takes an existing event list and either inserts a given event_map and returns True, or 
# decides that the event is a duplicate and returns false
def insert_event(event_list, event_map):
    # Find the index of the element already in the list which precedes the date in event_map.  
    # If not found, match_index is -1, so we should insert it at the beginning of event_list and return True
    # If found, check if the record matches what we already have (TODO: see if we need something
    # more sophisticated than ==).  If it matches what we already have, return False and do not modify the list.
    # If it doesn't match what we have insert it.  Note that list.insert takes the arg of the element to insert
    # before, so we use match_index+1
    match_index = find_event_index_by_date(event_list, event_map['date'])
    if(match_index==-1):
        #print "No match, insert at beginning"
        # No match.  Insert at the beginning, return True
        event_list.insert(0, event_map)
        return True
    # We've got a matching or preceeding event, compare it with this and any other potentially 
    # simultaneous events
    #print "Time match at %d" % (match_index)
    for i in range(match_index, len(event_list)):
        if(event_map == event_list[i]):
            # This is a duplicate, ignore it
            #print "Found duplicate match at %d (%r)" % (i,event_list[i])
            return False
        elif(event_map['date'] == event_list[i]['date'] and 'asofdate' in event_map and 'asofdate' in event_list[i] and event_map['asofdate'] == event_list[i]['asofdate']):
            # This is an event from the same sale date and asofdate, but it doesn't match exactly.
            # Have the new event supercede the old one.  Return true.
            #print "Found superceded version at %d (%r -> %r)" % (i,event_list[i], event_map)
            event_list[i]=event_map
            return True
        elif(event_map['date']<event_list[i]['date'] or ((event_map['date'] == event_list[i]['date'] and 'asofdate' in event_map and 'asofdate' in event_list[i] and event_map['asofdate'] < event_list[i]['asofdate']))):
            # We're past any simultaneous events that might have matched, continue
            # Note that this version only works if the asofdates within a given sale date 
            # are always kept sorted
            #print "Hit a later item at %d (%r)" % (i,match_index)
            break
        #print "Didn't match at %d (%r), keep trying" % (i,event_list[i])
        match_index=i
    # We've got a preceeding or simultaneous event that doesnt match.  Insert this 
    # new item just past match_index
    #print "Inserting after %d" % (match_index)
    event_list.insert(match_index+1, event_map)
    return True

# This takes an existing event list and either deletes a given event_map and returns True, or 
# decides that the event is not there and returns False
def delete_event(event_list, event_map):
    # Find the index of the element already in the list which matches or precedes the date in event_map.  
    # If not found, match_index is -1, so we should return False
    # If found, check if the record matches what we already have.  If it matches, delete and return True
    match_index = find_event_index_by_date(event_list, event_map['date'])
    if(match_index==-1 or event_map['date']>event_list[match_index]['date']):
        #print "No match"        
        return False
    # We've got a matching date for this event, compare it with this and any other potentially 
    # simultaneous events
    #print "Time match at %d" % (match_index)
    for i in range(match_index, len(event_list)):
        if(event_map == event_list[i]):
            #print "Found match at %d (%r), deleting" % (i,event_list[i])
            del event_list[i]
            return True
        elif(event_map['date'] == event_list[i]['date'] and 'asofdate' in event_map and 'asofdate' in event_list[i] and event_map['asofdate'] == event_list[i]['asofdate']):
            # This is an event from the same sale date and asofdate, delete and return true.
            #print "Found matching date and asofdate at %d, deleting" % (i)
            del event_list[i]
            return True
        elif(event_map['date']<event_list[i]['date']):
            # We're past any simultaneous events that might have matched, break and return False
            break
    #print "No match"
    return True

# Create 'property_address_raw' and 'owner_address_raw' columns.  Call once per dataframe before calling
# canonicalize_addrs
def merge_addr_colset(df, col_arr, out_colname):
    for  col_name in col_arr:
        df[col_name]=df[col_name].astype(basestring)
        df[col_name].fillna('', inplace=True)

    df[out_colname] = df[col_arr].apply(lambda x: ' '.join(x), axis=1)
    # Fix up PGH to be PITTSBURGH
    df[out_colname] = df[out_colname].str.replace('\s*PGH\s*,', ' PITTSBURGH,', regex=True)
    # Fix up to make sure there aren't differences in whitespace that could be avoided
    df[out_colname] = df[out_colname].str.replace('\s\s+', ' ', regex=True)
    
def merge_addrs(df):
    property_addr_cols=['PROPERTYHOUSENUM', 'PROPERTYADDRESS', 
                        'PROPERTYCITY' ,'PROPERTYSTATE', 'PROPERTYZIP']
    owner_addr_cols=['CHANGENOTICEADDRESS1', 'CHANGENOTICEADDRESS2',
                     'CHANGENOTICEADDRESS3', 'CHANGENOTICEADDRESS4']
    merge_addr_colset(df, property_addr_cols, 'property_address_raw')
    merge_addr_colset(df, owner_addr_cols, 'owner_address_raw')

# This assumes that merge_addrs has already been called to create the columns 'property_address_raw' and 
# 'owner_address_raw'
def canonicalize_cache_addr_col(df, colname):
    chunk_size=100
    total_len = len(df)
    next_i=0
    chunk_start_time=arrow.now()
    print_subset=10
    print_count=0
    retry_num=0
    while(next_i<total_len):
        u_addrs = df[colname][next_i:next_i+chunk_size].unique()
        if((print_count % print_subset) == 0):
            print "Processing %s for rows %d-%d, %d unique addresses (p=%d)" % (colname, next_i,next_i+chunk_size-1, len(u_addrs),print_count)
        try:
            cache_canonical_addresses(u_addrs)
            if((print_count % print_subset) == 0):
                print "   %s time elapsed" % (arrow.now()-chunk_start_time)
            next_i=next_i+chunk_size
            print_count=print_count+1
            chunk_start_time=arrow.now()
            retry_num = 0
        except:
            print "Error while processing %s for rows %d-%d, %d unique addresses (p=%d), retry %d: %s" % (colname, next_i,next_i+chunk_size-1, len(u_addrs),print_count, retry_num, sys.exc_info()[0])
            if(retry_num>=5):
                # Give up on that block
                print "   Giving up on that block"
                next_i=next_i+chunk_size
                print_count=print_count+1
                chunk_start_time=arrow.now()
            else:
                # Try again
                retry_num = retry_num+1
                
  
def canonicalize_addrs(df):
    if(not 'property_address_raw' in list(df) or not 'owner_address_raw' in list(df)):
        print "ERROR: No 'property_address_raw' or 'owner_address_raw' column, make sure merge_addrs has been run on this dataframe"
        raise
        
    # First make sure all the addresses are in the cache
    canonicalize_cache_addr_col(df, 'property_address_raw')
    canonicalize_cache_addr_col(df, 'owner_address_raw')
    print "* Address canonicalization cacheing complete.  Harvesting property addresses"
    # Harvest results.  They should all be in the cache
    df['property_address'] = df['property_address_raw'].apply(get_canonical_address_census)
    print "* Harvesting owner addresses"
    df['owner_address'] = df['owner_address_raw'].apply(get_canonical_address_census)
    print "* Done"

# Clean up multiple whitespace to a single space
def cleanup_str(str_in):
    str_out = re.sub( '\s+', ' ', str_in).strip()
    return(str_out)

# Try to normalize changeaddr to be more likely to match
re_zp4 = re.compile('(.+) (\d\d\d\d\d)-(\d\d\d\d)?')
re_cscz = re.compile('(.+) (\w\w), (\d\d\d\d\d)')
# Check for change address with no alphanumerics in it
re_noalnum = re.compile('^([^\w]+)$')

# The raw_owner_changeaddr is the one that should be used for detecting multiple ownership since it
# preserves unit number.  The canonical version loses it.
def get_raw_owner_changeaddr(apd, i):
    try:
        return(apd['owner_address_raw'].iloc[i])
    except:
        print "ERROR: No 'owner_address_raw' column, make sure merge_addrs has been run on this dataframe"
        raise

def get_canonical_owner_changeaddr(apd, i):
    try:
        return(apd['owner_address'].iloc[i])
    except:
        print "ERROR: No 'owner_address' column, make sure canonicalize_addrs has been run on this dataframe"
        raise

def update_property_address_info(parid, raw_addr, property_address):
    global canonical_property_address_map
    global raw_property_address_map

    # Check if the canonical property address is cached
    if(parid in canonical_property_address_map and canonical_property_address_map[parid]):
        return False
    # Not cached, see if we should add it
    if(property_address):
        # If this is a valid property address, cache it for next time
        canonical_property_address_map[parid]={'census':property_address}
        # Also cache the raw address
        raw_property_address_map[parid]=raw_addr
        #print "Cacheing raw address for %r: %r" % (parid, raw_addr)
    else:
        # Don't have a valid canonical address, only cache the raw address if we don't have 
        # anything in there yet.  
        if(raw_addr and parid not in raw_property_address_map):
            raw_property_address_map[parid]=raw_addr
            #print "Cacheing raw address for %r (can't canonicalize): %r" % (parid, raw_addr)
    return True

def update_property_address(apd, i):
    parid=apd.index[i]
    # Check if the canonical property address is cached
    if(parid in canonical_property_address_map and canonical_property_address_map[parid]):
        return canonical_property_address_map[parid]
    # Not cached, try to retrieve from data frame
    try:
        ret=apd['property_address'].iloc[i]
        raw_addr=apd['property_address_raw'].iloc[i]
        if(ret):
            # If this is a valid property address, cache it for next time
            canonical_property_address_map[parid]=ret
            # Also cache the raw address
            if(raw_addr):
                raw_property_address_map[parid]=raw_addr
                #print "Cacheing raw address for %r: %r" % (parid, raw_addr)
        else:
            # Don't have a valid canonical address, only cache the raw address if we don't have 
            # anything in there yet.  Return None
            if(raw_addr and parid not in raw_property_address_map):
                raw_property_address_map[parid]=raw_addr
                #print "Cacheing raw address for %r (can't canonicalize): %r" % (parid, raw_addr)
            return None
    except:
        #print "ERROR: No 'property_address' column, make sure canonicalize_addrs has been run on this dataframe"
        #raise
        # It's ok to not have a property_address, as some of the old versions are hopelessly messed up for that
        pass

def get_canonical_property_addr(parid):
    # Check if the canonical property address is cached
    if(parid in canonical_property_address_map.keys()):
        return canonical_property_address_map[parid]
    # Nope.  Return None
    return None

# First arg is a pandas table, second is an index for what to process.
# This updates property_map, owner_map, and changeaddr_map appropriately
# Returns True on success, False on failure
def process_assessment_record(apd, i, asofdate):
    global canonical_property_address_map
    global raw_property_address_map
    global property_map
    global owner_map
    global changeaddr_map

    # Generate set of interesting colnames we have in this data frame
    interesting_colnames=set(['SALEDESC','SALEPRICE','HOMESTEADFLAG','HOMESTEADFLAG',
                              'USEDESC','FAIRMARKETTOTAL','FAIRMARKETTOTAL',
                              'is_vacant','property_address_raw','owner_address_raw',
                              'property_address','owner_address'])
    colnames = set(apd.columns) & interesting_colnames
    #print "This dataframe has interesting columns %r" % (colnames)
    
    # Extract PARID, saledate, owner_name, and changeaddr from the record
    par_id = apd.index[i]
    
    # Process saledate
    saledate_raw = apd['SALEDATE'].iloc[i]
    # If saledate is valid, keep it.  Otherwise return False
    if(saledate_raw == '' or (isinstance(saledate_raw, numbers.Number) and math.isnan(saledate_raw))):
        #print "%s: Missing saledate" % (par_id)
        return False
    
    # Convert saledatet YMD so it sorts properly 
    saledate = SaledateToYMD(saledate_raw)
    
    owner_name = cleanup_str(apd['PROPERTYOWNER'].iloc[i])
    # Use the raw changeaddr for computing multiple ownership since it preserves
    # unit number.  Canonical doesn't 
    owner_changeaddr = get_raw_owner_changeaddr(apd,i)
        
    # If owner_changaddr is None or doesn't contain alphanumeric characters, skip 
    # TODO: Consider including entries with an empy changeaddr since it could help with accuracy 
    # of SALE calculations for the owners 
    if(pandas.isnull(owner_changeaddr) or re_noalnum.match(owner_changeaddr)):
        #print "%s: Missing owner_changeaddr" % (par_id)
        # skip this one
        return False
    
    # We have valid par_id, owner_name, and owner_changeaddr.  Update the maps.
    property_event={'date': saledate, 'asofdate': asofdate, 
                    'event_type':'PURCHASE', 'ownername':owner_name, 'changeaddr': owner_changeaddr}
    owner_event={'date':saledate, 'event_type':'PURCHASE', 'parid':par_id}

    # Check for nice-to-have columns, if valid value, add to property_event
    try:
        if('SALEDESC' in colnames):
            saledesc = apd['SALEDESC'].iloc[i]
            if(saledesc != '' and not (isinstance(saledesc, numbers.Number) and math.isnan(saledesc))):
                property_event['saledesc']=saledesc
                
        if('SALEPRICE' in colnames):
            saleprice = apd['SALEPRICE'].iloc[i]
            if(saleprice != '' and not (isinstance(saleprice, numbers.Number) and math.isnan(saleprice))):
                property_event['saleprice']=saleprice

        if('HOMESTEADFLAG' in colnames):
            homesteadflag = apd['HOMESTEADFLAG'].iloc[i]
            if(homesteadflag != '' and not (isinstance(homesteadflag, numbers.Number) and math.isnan(homesteadflag))):
                property_event['homesteadflag']=homesteadflag

        if('OWNERDESC' in colnames):
            ownerdesc = apd['OWNERDESC'].iloc[i]
            if(ownerdesc != '' and not (isinstance(ownerdesc, numbers.Number) and math.isnan(ownerdesc))):
                property_event['ownerdesc']=ownerdesc

        if('USEDESC' in colnames):
            usedesc = apd['USEDESC'].iloc[i]
            if(usedesc != '' and not (isinstance(usedesc, numbers.Number) and math.isnan(usedesc))):
                property_event['usedesc']=usedesc

        if('FAIRMARKETTOTAL' in colnames):
            fm_total = apd['FAIRMARKETTOTAL'].iloc[i]
            if(fm_total != '' and not (isinstance(fm_total, numbers.Number) and math.isnan(fm_total))):
                property_event['fm_total']=fm_total

        if('is_vacant' in colnames):
            is_vacant = apd['is_vacant'].iloc[i]
            if(not math.isnan(fm_total)):
                property_event['is_vacant']=is_vacant
    except:
        pass

    try:
        # Cache the canonical address for this property if available
        update_property_address(apd, i)
        # Store the canonical owner address with this property record if available
        canonical_owner_addr=get_canonical_owner_changeaddr(apd, i)
        if(canonical_owner_addr):
            property_event['canonical_owner_address']={'census':canonical_owner_addr}
    except:
        pass

    #print "-------\nProcessing %s: %r %r %r" % (par_id, saledate, owner_name, owner_changeaddr)

    if(not property_map.has_key(par_id)):
        # We don't have an entry for this property yet, create an empty list
        property_map[par_id]=[]
    
    # Insert the property event
    pret = insert_event(property_map[par_id], property_event)
 
    if(not owner_map.has_key(owner_name)):
        # We don't have an entry for this owner name yet, create an empty list
        owner_map[owner_name]=[]
    
    # Insert the owner event to owner map
    oret = insert_event(owner_map[owner_name], owner_event)
   
    if(not changeaddr_map.has_key(owner_changeaddr)):
        # We don't have an entry for this changeaddr yet, create an empty list
        changeaddr_map[owner_changeaddr]=[]
    
    # Insert the owner event
    cret = insert_event(changeaddr_map[owner_changeaddr], owner_event)
    
    # Print diagnostics
    #print "%s: %r %r %r = %r %r %r" % (par_id, saledate, owner_name, owner_changeaddr, pret, oret, cret)
    return(pret or oret or cret)

# Clear out the cache that stores what sets of properties are associated with each owner changeaddr at a given time.  
# We need to do this whenever we process in a new dataset.
def reset_cache():
    owned_parids_cache={}
    
# For a common parcel id, check if two events are both for the same transaction.  
# This happens when the dates are the same and/or when the set of owner names overlap
def is_same_transaction(e1,e2):
    if(e1['event_type']!='PURCHASE' or e2['event_type']!='PURCHASE'):
        return False
    if(e1['date']==e2['date']):
        return True
    if((e1['ownername'] in e2['ownername']) or (e2['ownername'] in e1['ownername'])):
        return True
    return False
    
def register_sale_dates(parid):
    prop_ev_list = property_map[parid]
    if(len(prop_ev_list)<2):
        # No sales have happened that we know about, only purchases
        return

    # Keep a list of the equivalent transactions so any subsequent sale can 
    # be applied to all.  Seed it with the 0'th item in prop_ev_list
    equiv_transactions=[prop_ev_list[0]]
    #print "Starting with %r" %(equiv_transactions)
    for i in range(1,len(prop_ev_list)):
        # See if this new transaction is equivalent to the last item in the current list
        if(is_same_transaction(prop_ev_list[i],equiv_transactions[-1])):
            equiv_transactions.append(prop_ev_list[i])
            #print "  %d still equivalent %r" %(i, equiv_transactions)
        else:
            #print "  %d not equivalent %r, record sale date" %(i,prop_ev_list[i])
            sell_event={'date':prop_ev_list[i]['date'],'event_type':'SALE','parid':parid}
            
            for j in range(0,len(equiv_transactions)):
                    dup_o = insert_event(owner_map[equiv_transactions[j]['ownername']], sell_event)
                    dup_c = insert_event(changeaddr_map[equiv_transactions[j]['changeaddr']], sell_event)
                    #print "    %d %r: %r %r" % (j, equiv_transactions[j], dup_o, dup_c)
            # Reset equiv_transactions and keep going if any more transactions are left
            equiv_transactions=[prop_ev_list[i]]
        
# This needs to be called once after imports are complete and before get_owned_parids
def register_all_sales():
    # Reset the owner and changeaddr caches since this will change the results
    reset_cache()
    
    # Add SALE records each time a property goes into different hands
    for parid in property_map.keys():
        register_sale_dates(parid)
   
def get_next_sale_date(parid, start_date):
    event_list = property_map[parid]
    # Get the index of the first transaction at or before this date
    start_index=find_event_index_by_date(event_list, start_date)

    # Starting with the transaction at or before the initial start date, 
    # go past all the equivalent transactions to the next one that's a real transfer.
    # If no future transfers, then return None
    equiv_transactions=[event_list[start_index]]

    for i in range(start_index,len(event_list)):
        # See if this new transaction is equivalent to the last item in the current list
        if(is_same_transaction(event_list[i],equiv_transactions[-1])):
            equiv_transactions.append(event_list[i])
            #print "  %d still equivalent %r" %(i, equiv_transactions)
        else:
            # This is a real tranfer, return the event date
            return event_list[i]['date']
    # If we get to here, there is no event after this date.  Return None
    return None

    
def get_owned_parids(name, event_list, eval_date):
    # Check if this name:date pair is in the cache.  If so, return the 
    # stored map of {'parids':owned_parids,'next_date':next_date}
    name_date_str=u"%s:%s"%(name,eval_date)
    if(name_date_str in owned_parids_cache):
        return(owned_parids_cache[name_date_str])

    # Return value wasn't in the cache, compute it
    
    # Use a set for accumulating parids since we don't want duplicates
    owned_parids=set()
    
    # In case eval_date is at or after the last date in the list, 
    # default next_date to today
    next_date = this_date
    for i in range(0,len(event_list)):
        if(event_list[i]['date']<= eval_date):
            # This event happened on or before the date we're asking about, process the event
            if(event_list[i]['event_type']=='PURCHASE'):
                # Add purchased property to owned_parids
                owned_parids.add(event_list[i]['parid'])
            else:
                # This must be a sale, remove it from owned_parids
                # Note that this can potentially fail in the case where 
                # two subsequent sales of the same property involve the same 
                # change address if the transactions happen as (add, 
                # (add, remove) from the same day, remove).  This actually 
                # happens with PARID == '0104R00158000000'.  So, we put this
                # in a try/catch block.  Bleh...
                try:
                    owned_parids.remove(event_list[i]['parid'])
                except:
                    pass
        else:
            # This and subsequent events happened after the date we're looking for.
            # Set next_date and return owned_parids
            next_date = event_list[i]['date']
            break

    ret_val = {'parids':owned_parids,'next_date':next_date}
    # Store in cache for next time
    owned_parids_cache[name_date_str]=ret_val
    return ret_val

# For a given parid and eval_date, get a map back containing:
#   'parids' = the set of parids owned by the same owners/changeaddrs as of eval_date, and
#   'next_date' = the next date that something changes within that set of owners/changeaddrs
# When the last of the related events is complete, 'next_date'==this_date
def get_related_parids(parid, eval_date):
    prop_ev_list = property_map[parid]
    parid_set = set()
    start_date = eval_date
    next_date = this_date

    # Keep a list of the equivalent transactions so we can accumulate the 
    # parids for all.  Seed it with the 0'th item in prop_ev_list
    equiv_transactions=[prop_ev_list[0]]
    # If date of the first event is later than eval_date, set eval_date to 
    # the first event.  We don't want to iterate over the earlier purchases by
    # the original owners we know about.  Just start at the first purchase
    # of this parcel we know about.
    if(eval_date<prop_ev_list[0]['date']):
        eval_date=prop_ev_list[0]['date']
        start_date=eval_date
    #print "%s: starting with %r" %(parid, equiv_transactions)
    for i in range(1,len(prop_ev_list)):
        # See if this new transaction is equivalent to the last item in the current list
        if(is_same_transaction(prop_ev_list[i],equiv_transactions[-1]) and 
           prop_ev_list[i]['date']<=eval_date):
            equiv_transactions.append(prop_ev_list[i])
            #print "  %d still equivalent %r" %(i, equiv_transactions)
        elif(prop_ev_list[i]['date']>eval_date):
            #print "  %d after timespan (%r), process current set" %(i,prop_ev_list[i])
            break
        else:
            # We haven't hit the end of the timespan yet, but the ownership has changed.
            # Reset equiv_transactions starting from the current event
            equiv_transactions=[prop_ev_list[i]]
            
    # We've got all the equivalent transactions, process them
    changeaddr_set=set()
    owner_set=set()
    for j in range(0,len(equiv_transactions)):
        owner_set.add(equiv_transactions[j]['ownername'])
        changeaddr_set.add(equiv_transactions[j]['changeaddr'])

    #print "    Owner set= %r\n    Changeaddr set= %r" % (owner_set, changeaddr_set)
    
    for owner in owner_set:
        ret_map=get_owned_parids(owner, owner_map[owner],eval_date)
        parid_set = parid_set.union(ret_map['parids'])
        if(ret_map['next_date']<next_date):
            next_date = ret_map['next_date']
    for changeaddr in changeaddr_set:
        ret_map=get_owned_parids(changeaddr, changeaddr_map[changeaddr],eval_date)
        parid_set = parid_set.union(ret_map['parids'])
        if(ret_map['next_date']<next_date):
            next_date = ret_map['next_date']

    ret_val={"parids":parid_set, "parcount":len(parid_set), "start_date":start_date, "next_date":next_date}
    #print "    Returning %r" % (ret_val)
    return ret_val

def get_simultaneous_property_events(prop_ev_list, start_index):
    eval_date=prop_ev_list[start_index]['date']
    start_date = eval_date
    next_date = this_date

    # Keep a list of the equivalent transactions.  Seed it with the start_index'th item in prop_ev_list
    equiv_transactions=[prop_ev_list[start_index]]
    # Check if this is the last transaction in prop_ev_list
    if(start_index+1>=len(prop_ev_list)):
        return equiv_transactions
    
    # At least one left
    for i in range(start_index+1,len(prop_ev_list)):
        # See if this new transaction is equivalent to the last item in the current list
        if(is_same_transaction(prop_ev_list[i],equiv_transactions[-1]) and 
           prop_ev_list[i]['date']==eval_date):
            equiv_transactions.append(prop_ev_list[i])
            #print "  %d still equivalent %r" %(i, equiv_transactions)
        elif(prop_ev_list[i]['date']>eval_date):
            #print "  %d after timespan (%r), process current set" %(i,prop_ev_list[i])
            break
        else:
            # We haven't hit the end of the timespan yet, but the ownership has changed.
            # Reset equiv_transactions starting from the current event
            break
            
    # We've got all the equivalent transactions, sort them according to asofdate
    return sorted(equiv_transactions, key=lambda ev: ev['asofdate'])

# Call this once for each assessment spreadsheet/database file.  It adds PURCHASE records if they don't already exist.
# It's safe to call multiple times on the same input file if necessary, but takes a lot of processing time
# asofdate, in format YYYY-MM-DD, is used to disambiguate earlier vs later versions of addresses or values
def process_all_assessment_records(apd, asofdate):
    start=arrow.now()
    addcnt=0
    chunk_start_time=arrow.now()
    chunk_size=10000

    # Clear the owner and changeaddr caches as this will cause changes in the ownership profiles
    reset_cache()
    
    # Iterate over each record in the data frame and add to the maps
    for i in range(0,len(apd)):
        if(process_assessment_record(apd,i, asofdate)):
            addcnt=addcnt+1
        if((i%chunk_size)==0 and i>0):
            print "%d-%d: %d added, %s time elapsed" %(i-(chunk_size-1), i, addcnt, arrow.now()-chunk_start_time)
            #break
            addcnt=0
            chunk_start_time=arrow.now()
    end=arrow.now()
    print "Processing took %s" % (str(end-start))

# Support save/restore for assessment maps

In [ ]:
# Try to save out property_map before we start messsing with it
import pickle
pickle_file_t = 'assessments/{0}_{1}.pickle'

# Save this data frame using a suffix to get it back with.  
# For example, '99_04_05_09_17'
def save_assessment_maps(suffix):
    global property_map
    global owner_map
    global changeaddr_map
    global canonical_property_address_map
    global raw_property_address_map

    print "** Saving assessment maps with suffix %s" % (suffix)
    
    with open(pickle_file_t.format('property_map',suffix), 'wb') as handle:
        pickle.dump(property_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(pickle_file_t.format('owner_map',suffix), 'wb') as handle:
        pickle.dump(owner_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(pickle_file_t.format('changeaddr_map',suffix), 'wb') as handle:
        pickle.dump(changeaddr_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(pickle_file_t.format('canonical_property_address_map',suffix), 'wb') as handle:
        pickle.dump(canonical_property_address_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(pickle_file_t.format('raw_property_address_map',suffix), 'wb') as handle:
        pickle.dump(raw_property_address_map, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    with open(pickle_file_t.format('parid2centroid',suffix), 'wb') as handle:
        pickle.dump(parid2centroid, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print "** Done"
    
def restore_assessment_maps(suffix):
    global property_map
    global owner_map
    global changeaddr_map
    global canonical_property_address_map
    global raw_property_address_map
    
    print "** Restoring assessment maps with suffix %s" % (suffix)

    with open(pickle_file_t.format('property_map',suffix), 'rb') as handle:
        property_map = pickle.load(handle)
    print "    Loaded %d records from %s" % (len(property_map), pickle_file_t.format('property_map',suffix))

    with open(pickle_file_t.format('owner_map',suffix), 'rb') as handle:
        owner_map = pickle.load(handle)
    print "    Loaded %d records from %s" % (len(owner_map), pickle_file_t.format('owner_map',suffix))
    
    with open(pickle_file_t.format('changeaddr_map',suffix), 'rb') as handle:
        changeaddr_map = pickle.load(handle)
    print "    Loaded %d records from %s" % (len(changeaddr_map), pickle_file_t.format('changeaddr_map',suffix))
    
    with open(pickle_file_t.format('canonical_property_address_map',suffix), 'rb') as handle:
        canonical_property_address_map = pickle.load(handle)
    print "    Loaded %d records from %s" % (len(canonical_property_address_map), pickle_file_t.format('canonical_property_address_map',suffix))
    
    with open(pickle_file_t.format('raw_property_address_map',suffix), 'rb') as handle:
        raw_property_address_map = pickle.load(handle)
    print "    Loaded %d records from %s" % (len(raw_property_address_map), pickle_file_t.format('raw_property_address_map',suffix))
    print "** Done"



In [ ]:
restore_assessment_maps('pms_cac_99b_04d_05_09_10_11_17_18b')

In [ ]:
apd_15217_2017 = apd_2017[apd_2017.PROPERTYZIP=='15217'].copy()

In [ ]:
merge_addrs(apd_15213_2017)

In [ ]:
canonicalize_addrs(apd_15213_2017)

In [ ]:
process_all_assessment_records(apd_15213_2017,'2017-10-01')

In [ ]:
# merge property and owner addresses
merge_addrs(apd_2017)

In [ ]:
canonicalize_addrs(apd_2017)

In [ ]:
# Process entries from apd_2017 into the property, owner, and changeaddr maps
process_all_assessment_records(apd_2017, '2017-10-01')

In [ ]:
len(apd_2017)

In [ ]:
# Save out apd_2017 to mdf
save_adf(apd_2017, 'apd_2017')

In [ ]:
# The following are historical fixups I had to do for the 2017 data set, but which shouldn't need to be done again

In [ ]:
apd_2017.loc[['0174E00236000000','0174E00234000000','0029H00025000000']]

In [ ]:
vacant_col1_2017 = (apd_2017.FAIRMARKETBUILDING<2.0)
vacant_col2_2017 = ((apd_2017['USEDESC'].fillna('').str.contains('VACANT')) | (apd_2017['USEDESC'].fillna('').str.contains('UNLOCATED PARCEL')) | (apd_2017['USEDESC'].fillna('').str.contains('CONDEMNED')))
vacant_col_2017=(vacant_col1_2017 | vacant_col2_2017)
apd_2017[vacant_col_2017]

In [ ]:
vacant_col_2017.name='is_vacant'
apd_2017 = pd.concat([apd_2017, vacant_col_2017], axis=1)

## Process 2018

In [ ]:
# Restore apd_2018 from mdf if we don't want to reprocess from scratch
apd_2018 = restore_adf('apd_2018')

In [ ]:
# Read in assessments data as TDF, set index to PARID
path = "assessments/2018-01-cd/AC Property Assessments_01222018.xls"
raw_pd_2018 = pd.read_csv(path,sep='\t', index_col='PARID',dtype={'PROPERTYHOUSENUM':numpy.str,'CHANGENOTICEADDRESS4':numpy.str, 'PROPERTYZIP':numpy.str})

In [ ]:
# Consider a parcel a dwelling if either residential or apartments (usedesc includes APART)
# dwelling_col is indexed on PARID, has TRUE for dwellings, and FALSE for non-dwellings
dwelling_col_2018 = ((raw_pd_2018.CLASSDESC == 'RESIDENTIAL') | (raw_pd_2018.USEDESC.str.contains('APART')))
dwelling_col_2018.name='is_dwelling'

# Only keep the subset that are dwellings
apd_2018 = pd.concat([raw_pd_2018, dwelling_col_2018], axis=1)[dwelling_col_2018]
len(apd_2018)

In [ ]:
apd_2018.loc[['0174E00236000000','0174E00234000000','0029H00025000000']]

In [ ]:
vacant_col1_2018 = (apd_2018.FAIRMARKETBUILDING<2.0)
vacant_col2_2018 = ((apd_2018['USEDESC'].fillna('').str.contains('VACANT')) | (apd_2018['USEDESC'].fillna('').str.contains('UNLOCATED PARCEL')) | (apd_2018['USEDESC'].fillna('').str.contains('CONDEMNED')))
vacant_col_2018=(vacant_col1_2018 | vacant_col2_2018)
apd_2018[vacant_col_2018]

In [ ]:
vacant_col_2018.name='is_vacant'
apd_2018 = pd.concat([apd_2018, vacant_col_2018], axis=1)

In [ ]:
# merge property and owner addresses
merge_addrs(apd_2018)
canonicalize_addrs(apd_2018)


In [ ]:
apd_2018

In [ ]:
property_map['0002M00197000000']

In [ ]:
process_assessment_record(apd_2018, 0, '2018-01-22')

In [ ]:
colnames = ['owner_address', 'is_vacant', 'HOMESTEADFLAG', 'USEDESC', 'property_address_raw', 'SALEPRICE', 'FAIRMARKETTOTAL', 'SALEDESC', 'owner_address_raw', 'property_address']

In [ ]:
%%timeit
'is_vacant' in colnames

In [ ]:
# Save out apd_2018 to mdf
save_adf(apd_2018, 'apd_2018')

In [ ]:
# Process entries from apd_2018 into the property, owner, and changeaddr maps
process_all_assessment_records(apd_2018, '2018-01-22')

# Deal with older versions of the assessment database from Kristin

In [ ]:
# Original processing of 1999 data is in
#   http://localhost:8815/notebooks/uwsgi/dotmaptiles/data-visualization-tools/examples/lodes/Import%20DBF%20Test.ipynb
#
# It was then pulled into a copy of this notebook and processed:
#  apd_1999 = merge_dwelling_col(apd_1999)
#  pa_2017 = apd_2017[['property_address_raw','property_address']]
#  # Merge property_address_raw and property_address into 1999
#  apd_1999 = pd.merge(adf_1999,pa_2017,on='PARID', left_on=None, right_on=None,
#                left_index=False, right_index=False, sort=False,
#                suffixes=('_x', '_y'), copy=True, indicator=False,
#                validate=None)
#  # After canonicalization cacheing completes
#  apd_1999['owner_address'] = apd_1999['owner_address_raw'].apply(get_canonical_address)
# 
#  Manual fixup of bad addresses and sale dates:
#  apd_1999.set_value('0162H00002000000','property_address_raw','124 MONTVILLE ST PITTSBURGH PA 15214')
#  apd_1999.set_value('0162H00002000000','property_address',get_canonical_address('124 MONTVILLE ST PITTSBURGH PA 15214'))
# 
#  apd_1999.set_value('0232C00201000000','SALEDATE','1970-11-01')
#  The result was saved out using:
#  save_adf(apd_1999, 'apd_1999')
#
apd_1999 = restore_adf('apd_1999')

In [ ]:
# Process assessment records from apd_1999.  We don't know when it's from exactly, but the most recent 
# SALEDATE in there is '1997-12-01' (apd_1999['SALEDATE'].max()), so '1999-01-01' seems safe
process_all_assessment_records(apd_1999, '1999-01-01')

In [ ]:
# The following are historical fixups I had to do for the 1999 data set, but which shouldn't need to be done again

In [ ]:
# Do fixup to create FAIRMARKETTOTAL column.  This only needs to be done once.
col_rename_1999_augment = {'PRICELASTS':'SALEPRICE',
'TLANDASMT':'FAIRMARKETLAND',
'TBLDGASMT':'FAIRMARKETBUILDING'}

apd_1999= apd_1999.rename(index=str, columns=col_rename_1999_augment)

# Construct a 'FAIRMARKETTOTAL' column
apd_1999['FAIRMARKETTOTAL']=apd_1999[['FAIRMARKETLAND','FAIRMARKETBUILDING']].apply(sum, axis=1)


vacant_col_1999 = (apd_1999.FAIRMARKETBUILDING==0.0)
vacant_col_1999.name='is_vacant'
apd_1999 = pd.concat([apd_1999, vacant_col_1999], axis=1)
apd_1999

# Save apd_1999 back out
save_adf(apd_1999, 'apd_1999')

In [ ]:
import pprint
pp = pprint.PrettyPrinter()
    
def change_property_event_date(parid, orig_date, asofdate, new_date):
    event_list = property_map[parid]
    # Find the index of the element already in the list which matches or precedes orig_date.  
    # If not found, match_index is -1, so we should return False
    # If found, check if the record matches what we already have.  If it matches, delete and return True
    match_index = find_event_index_by_date(event_list, orig_date)
    if(match_index==-1 or orig_date>event_list[match_index]['date']):
        print "%s: No match" % (parid)
        return False
    # We've got a matching date for this event, compare it with this and any other potentially 
    # simultaneous events
    print "%s: Time match at %d" % (parid, match_index)
    for i in range(match_index, len(event_list)):
        if(orig_date == event_list[i]['date'] and 'asofdate' in event_list[i] and asofdate == event_list[i]['asofdate']):
            # This is an event from the same sale date and asofdate, delete, modify, reinsert, 
            # Modify changeaddr_map and owner_map, and return true.
            print "%s: Found matching date and asofdate in property_map at %d, modifying" % (parid, i)
            event_map = event_list[i]
            del event_list[i]
            event_map['date']=new_date
            res_eins = insert_event(event_list, event_map)
                            
            old_owner_event={'date':orig_date, 'event_type':'PURCHASE', 'parid':parid}
            new_owner_event={'date':new_date, 'event_type':'PURCHASE', 'parid':parid}
            res_cdel = delete_event(changeaddr_map[event_map['changeaddr']], old_owner_event)
            res_cins = insert_event(changeaddr_map[event_map['changeaddr']], new_owner_event)
            
            res_odel = delete_event(owner_map[event_map['ownername']], old_owner_event)
            res_oins = insert_event(owner_map[event_map['ownername']], new_owner_event)

            print "property_map insert %r:\n   property_map[%r]=" % ("SUCCEEDED" if res_eins else 'FAILED', parid)
            pp.pprint(property_map[parid])
            print "changeaddr_map del/ins %r\n   changeaddr_map[%r]=" % ("SUCCEEDED" if (res_cdel and res_cins) else 'FAILED', event_map['changeaddr'])
            pp.pprint(changeaddr_map[event_map['changeaddr']])
            print "owner_map del/ins %r\n   owner_map[%r]=" % ("SUCCEEDED" if (res_odel and res_oins) else 'FAILED', event_map['ownername'])
            pp.pprint(owner_map[event_map['ownername']])
            return True
        elif(orig_date<event_list[i]['date']):
            # We're past any simultaneous events that might have matched, break and return False
            break
    #print "No match"
    return False


In [ ]:
apd_1999[apd_1999.SALEDATE.str.contains('-00-')][['PROPERTYOWNER','SALEDATE']]

In [ ]:
property_map['0062G00184000000']

In [ ]:
change_property_event_date('0062G00184000000','1950-05-05', '2009-09-01','1956-05-21')

In [ ]:
change_property_event_date('0062G00184000000','1950-05-05', '1999-01-01','1956-05-21')

In [ ]:
change_property_event_date('0062G00184000000','1950-05-05', '2010-06-01','1956-05-21')

In [ ]:
change_property_event_date('0127N00014000000','1995-00-01', '1999-01-01', '1995-05-18')

In [ ]:
change_property_event_date('0120L00200000000','1955-00-01', '1999-01-01', '1955-01-01')

In [ ]:
len(changeaddr_map[u'539 S 4TH AVE LOUISVILLE KY 40202'])

In [ ]:
change_property_event_date('0009M00087000000', '1992-00-01', '1999-01-01', '1950-05-05')

In [ ]:
change_property_event_date('0009M00087000000', '1950-05-05', '1999-01-01', '1992-00-01')

In [ ]:
# Do fixup on bad addresses.  This should only have to be done once
apd_1999[apd_1999.SALEDATE.str.contains('-00-')][['PROPERTYOWNER','SALEDATE']]

In [ ]:
fixup_changeaddrs_from_asofdate(apd_1999.index, '1999-01-01')

In [ ]:
fixup_canonical_owner_address_from_asofdate(apd_1999.index, '1999-01-01')

In [ ]:
 len(changeaddr_map[u'539 S 4TH AVE LOUISVILLE KY 40202'])

In [ ]:
changeaddr_map[u'539 S 4TH AVE LOUISVILLE KY 40202'][1]

In [ ]:
property_map['0087R00101000000']

## 2004

In [ ]:
# Try to save out property_map before we start messsing with it
import pickle

with open('assessments/property_map_99_09_17.pickle', 'wb') as handle:
    pickle.dump(property_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
apd_2004 = restore_adf('apd_2004')

In [ ]:
len(apd_2004)

In [ ]:
# Process assessment records from apd_2004.  We don't know when it's from exactly, but the most recent 
# SALEDATE in there is '2001-02-15' (apd_2004['SALEDATE'].max()), so '2004-01-01' seems safe.
# Actually, comparing the 2004 to 2005 sets, we really need the asofdate to be '2001-02-15'
# instead of '2004-01-01'
process_all_assessment_records(apd_2004, '2001-02-15')

In [ ]:
# The following are historical fixups I had to do for the 2004 data set, but which shouldn't need to be done again

In [ ]:
# Rewrite property_map entries with asofdate of '2004-01-01' to '2001-02-15'
bad_asofdate='2004-01-01'
replacement_asofdate='2001-02-15'
count=0
for parid in apd_2004.index:
    for ev in property_map[parid]:
        if(ev['asofdate']==bad_asofdate):
            ev['asofdate']=replacement_asofdate
            if(count % 1000==0):
                print "%s: %s [%s] -> [%s]" % (parid, ev['date'], bad_asofdate, replacement_asofdate)
            count = count+1            

In [ ]:
for i in range(0, len(apd_2004)):
    update_property_address(apd_2004, i)

In [ ]:
count=0
for parid in noprop_parids_2004:
    if (not parid in canonical_property_address_map):
        print "%s missing from canonical_property_address_map (raw = %s)" % (parid, raw_property_address_map[parid])
    else:
        if((count % 100)==0):
            print "%d: Ok" % (count)
        count = count+1

In [ ]:
vacant_col_2004 = apd_2004.FAIRMARKETBUILDING==0.0
vacant_col_2004.name='is_vacant'
apd_2004 = pd.concat([apd_2004, vacant_col_2004], axis=1)

In [ ]:
# Deal with potential quality issue for owner addresses

In [ ]:
apd_2004.loc[['0174E00236000000','0174E00234000000','0029H00025000000']]

In [ ]:
apd_2004[apd_2004.owner_address_raw=='PO BOX 8469 CANTON OH 44711']

In [ ]:
fixup_changeaddrs_from_asofdate(apd_2004.index, '2001-02-15')

In [ ]:
fixup_canonical_owner_address_from_asofdate(apd_2004.index, '2001-02-15')

In [ ]:
len(changeaddr_map['PO BOX 8469 CANTON OH 44711'])

In [ ]:
property_map['9946X83377000000']

In [ ]:
find_event_index_by_date(changeaddr_map['133 JEFFERSON RD PITTSBURGH PA 15235'], '1981-08-19')

In [ ]:
owner_event={'date':'1981-08-19', 'event_type':'PURCHASE', 'parid':'0001C01662004200'}
delete_event(changeaddr_map['133 JEFFERSON RD PITTSBURGH PA 15235'],owner_event)

In [ ]:
changeaddr_map['PO BOX 22552 PITTSBURGH PA 15222']

In [ ]:
property_map[apd_2004.index[4]]

In [ ]:
save_adf(apd_2004,'apd_2004')

# 2005

In [ ]:
# Try to save out property_map before we start messsing with it
import pickle

with open('assessments/property_map_99_04_09_17.pickle', 'wb') as handle:
    pickle.dump(property_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
apd_2005 = restore_adf('apd_2005')

In [ ]:
# Process assessment records from apd_2005.  We don't know when it's from exactly, but the most recent 
# valid-looking SALEDATE in there is '2005-08-28 (apd_2005['SALEDATE'].max())
process_all_assessment_records(apd_2005, '2005-08-28')


In [ ]:
# The following are historical fixups I had to do for the 2005 data set, but which shouldn't need to be done again

In [ ]:
apd_2005.loc[['0174E00236000000','0174E00234000000','0029H00025000000']]

In [ ]:
vacant_col1_2005 = (apd_2005.FAIRMARKETBUILDING<2.0)
vacant_col2_2005 = ((apd_2005.USEDESC.fillna('').str.contains('VACANT')) | (apd_2005.USEDESC.fillna('').str.contains('UNLOCATED PARCEL')) | (apd_2005.USEDESC.fillna('').str.contains('CONDEMNED')))

In [ ]:
vacant_diff_col_2005=((vacant_col1_2005 & (~vacant_col2_2005)) | ((~vacant_col1_2005) & vacant_col2_2005))
apd_2005[vacant_diff_col_2005]

In [ ]:
vacant_col_2005=(vacant_col1_2005 | vacant_col2_2005)
vacant_col_2005.name='is_vacant'
apd_2005 = pd.concat([apd_2005, vacant_col_2005], axis=1)

# Deal with older versions of the assessment database from the ACCDB files from Bob Gradeck

In [ ]:
# From https://stackoverflow.com/questions/17123550/extract-and-sort-data-from-mdb-file-using-mdbtools-in-python?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
import sys, subprocess, os, numpy
from io import StringIO
import pandas as pd
import pandas
VERBOSE = True
def mdb_to_pandas(database_path,dtype_map):
    subprocess.call(["mdb-schema", database_path, "mysql"])
    # Get the list of table names with "mdb-tables"
    table_names = subprocess.Popen(["mdb-tables", "-1", database_path],
                                   stdout=subprocess.PIPE).communicate()[0]
    tables = table_names.splitlines()
    sys.stdout.flush()
    # Dump each table as a stringio using "mdb-export",
    out_tables = {}
    for rtable in tables:
        table = rtable.decode()
        if VERBOSE: print('running table:',table)
        if table != '':
            if VERBOSE: print("Dumping " + table)
            contents = subprocess.Popen(["mdb-export", database_path, table],
                                        stdout=subprocess.PIPE).communicate()[0]
            temp_io = StringIO(contents.decode('utf8'))
            print(table, temp_io)
            out_tables[table] = pd.read_csv(temp_io,dtype=dtype_map)
    return out_tables
pd.options.display.max_colwidth = 300
pd.set_option('display.max_columns', 500)


In [ ]:
# ACCDB files are from Bob Gradeck
# 2009 is missing YEARBLT, CLASSDESC, and CLASS

# A number of files after 2009 seem to have a common set of column names
post_2009_colmap = {'mapblolot':'PARID',
                    'PropertyHouseNum'    :'PROPERTYHOUSENUM', 
                    'PropertyFraction'    :'PROPERTYFRACTION', 
                    'PropertyAddress'     :'PROPERTYADDRESS',  
                    'PropertyUnit'        :'PROPERTYUNIT',     
                    'PropertyCityState'   :'PROPERTYCITY',     
                    'PropertyLocation2'   :'PROPERTYSTATE',    
                    'PropertyZip'         :'PROPERTYZIP',       
                    'FairMarketTotal':'FAIRMARKETTOTAL',
                    'UseDesc':'USEDESC',
                    'PropertyZip':'PROPERTYZIP',
                    'PropertyOwner2':'PROPERTYOWNER',
                    'ChangeNoticeFullAddress1':'CHANGENOTICEADDRESS1',
                    'ChangeNoticeFullAddress2':'CHANGENOTICEADDRESS2',
                    'ChangeNoticeFullAddress3':'CHANGENOTICEADDRESS3',
                    'ChangeNoticeFullAddress4':'CHANGENOTICEADDRESS4',
                    'HomesteadFlag':'HOMESTEADFLAG',
                    'OwnerDesc':'OWNERDESC',
                    'SaleDate':'SALEDATE',
                    'SalePrice':'SALEPRICE',
                    'SaleCode': 'SALECODE',
                    'SaleDesc': 'SALEDESC'
                    }
post_2009_dtype = {'PropertyHouseNum':numpy.str,'PropertyFraction':numpy.str, 'PropertyZip':numpy.str,
                    'ChangeNoticeFullAddress1':numpy.str,'ChangeNoticeFullAddress2':numpy.str, 
                    'ChangeNoticeFullAddress3':numpy.str,'ChangeNoticeFullAddress4':numpy.str, 
                   }

accdb_info = {'2004':{'fname': 'assessments/kristin/ParcelData_RandySargent/2004/db1.mdb',
                      'tname': 'LandFileAug2001',
                      'col_remap': {'MapBloLot':'PARID',
                                    'CurOwnerName':'PROPERTYOWNER',
                                    'Homestead':'HOMESTEADFLAG',
                                    'SalePrice':'SALEPRICE',
                                    'TotalLandAssess':'FAIRMARKETLAND',
                                    'TotalBuildingAssess':'FAIRMARKETBUILDING'
                        },
                      'dtype':{'MailStreetNumber':numpy.str,'MailZipCode':numpy.str, 
                               'ST_NUMBER':numpy.str, 'ZIPCODE':numpy.str
                              }
                     },
              '2005':{'fname': 'assessments/kristin/ParcelData_RandySargent/2005/AC db 10.20.05.mdb',
                      'tname': 'DS09-21-05',
                      'col_remap': {'Field1':'PARID',
                                    'Field2':'PROPERTYOWNER',
                                    'Field23':'OWNERDESC',
                                    'Field27':'USEDESC',
                                    'Field29':'HOMESTEADFLAG',
                                    'Field31':'SALEDATE_raw',
                                    'Field32':'SALEPRICE',
                                    'Field36':'FAIRMARKETBUILDING',
                                    'Field37':'FAIRMARKETLAND',
                                    'Field38':'FAIRMARKETTOTAL',
                                    'Field57':'SALECODE',
                                    'Field58':'SALEDESC',
                                    'Field62':'YEARRBLT'
                        },
                      'dtype':{'Field45':numpy.str,'Field31':numpy.str, 
                               'Field4' :numpy.str, 'Field5' :numpy.str, 'Field6' :numpy.str, 
                               'Field7' :numpy.str, 'Field8' :numpy.str, 'Field9' :numpy.str, 
                               'Field10':numpy.str, 'Field11':numpy.str
                              }
                     },
              '2009':{'fname':'assessments/gradeck/AssessmentSep09.mdb',
                      'tname':'pncis',
                     'col_remap': {'PIN':'PARID',
                                    'PROPERTYHO':'PROPERTYHOUSENUM', 
                                    'PROPERTYAD':'PROPERTYFRACTION', 
                                    'PROPERTY_1':'PROPERTYADDRESS',  
                                    'PROPERTYUN':'PROPERTYUNIT',     
                                    'PROPERTYLO':'PROPERTYCITY',     
                                    'PROPERTYCI':'PROPERTYSTATE',    
                                    'PROPERTYZI':'PROPERTYZIP',       
                                    'FAIRMARK_2':'FAIRMARKETTOTAL',
                                    'USEDESC':'USEDESC',
                                    'PROPERTYZI':'PROPERTYZIP',
                                    'PROPERTYOW':'PROPERTYOWNER',
                                    'CHANGENOTI':'CHANGENOTICEADDRESS1',
                                    'CHANGENO_1':'CHANGENOTICEADDRESS2',
                                    'CHANGENO_2':'CHANGENOTICEADDRESS3',
                                    'CHANGENO_3':'CHANGENOTICEADDRESS4',
                                    'HOMESTEADF':'HOMESTEADFLAG',
                                    'OWNERDESC':'OWNERDESC',
                                    'SALEDATE':'SALEDATE',
                                    'SALEPRICE':'SALEPRICE'
                                  },
                      'dtype':{'PROPERTYHO':numpy.str,'PROPERTY_1':numpy.str,'PROPERTYUN':numpy.str,
                                'PROPERTYLO':numpy.str,'PROPERTYCI':numpy.str,'PROPERTYZI':numpy.str}
                     },
               '2010':{'fname':'assessments/gradeck/June2010AssessNew.accdb',
                       'tname':'RawDataAssessment',
                       'col_remap': post_2009_colmap,
                       'dtype': post_2009_dtype
                      },
               '2011':{'fname':'assessments/gradeck/AssessMarch2011Data3.mdb.accdb',
                       'tname':'AssessMarch2011',
                       'col_remap': {'PIN':'PARID',
                                    'PropertyOwnerNew':'PROPERTYOWNER',
                                    'CHANGENOTICEFULLADDRESS1':'CHANGENOTICEADDRESS1',
                                    'CHANGENOTICEFULLADDRESS2':'CHANGENOTICEADDRESS2',
                                    'CHANGENOTICEFULLADDRESS3':'CHANGENOTICEADDRESS3',
                                    'CHANGENOTICEFULLADDRESS4':'CHANGENOTICEADDRESS4',
                                    'PROPERTYHOUSENUM'    :'PROPERTYHOUSENUM',
                                    'PROPERTYADDNUMPREFIX':'PROPERTYFRACTION',
                                    'PROPERTYADDRESS'     :'PROPERTYADDRESS',
                                    'PROPERTYUNIT'        :'PROPERTYUNIT',
                                    'PROPERTYLOCATION2'   :'PROPERTYCITY',
                                    'PROPERTYCITYSTATE'   :'PROPERTYSTATE',
                                    'PROPERTYZIP'         :'PROPERTYZIP'       

                                  },
                        'dtype':{'PROPERTYHOUSENUM':numpy.str,'PROPERTYADDRESS':numpy.str,'PROPERTYUNIT':numpy.str,
                                'PROPERTYLOCATION2':numpy.str,'PROPERTYCITYSTATE':numpy.str,'PROPERTYZIP':numpy.str}
                      },
               '2012':{'fname':'assessments/gradeck/AssessNov2012.mdb',
                       'tname':'AssessmentOct2_2012',                       
                       'col_remap': {'SALEDATE':'SALEDATE_raw'},
                       'dtype':{'PROPERTYHOUSENUM':numpy.str,'PROPERTYADDRESS':numpy.str,'PROPERTYUNIT':numpy.str,
                                'PROPERTYLOCATION2':numpy.str,'PROPERTYCITYSTATE':numpy.str,'PROPERTYZIP':numpy.str,
                                'SALEDATE':numpy.str, 'CHANGENOTICEADDRESS1':numpy.str,
                                'CHANGENOTICEADDRESS2':numpy.str, 'CHANGENOTICEADDRESS3':numpy.str,
                                'CHANGENOTICEADDRESS4':numpy.str,}
                      },
               '2013':{'fname':'assessments/gradeck/Dec2013Assess.accdb',
                       'tname':'RawDataAssessment',
                       'col_remap': post_2009_colmap,
                       'dtype': post_2009_dtype
                      },
               '2014':{'fname':'assessments/gradeck/June2014Assess.accdb',
                       'tname':'RawDataAssessment',
                       'col_remap': post_2009_colmap,
                       'dtype': post_2009_dtype
                      },
               '2015':{'fname':'assessments/gradeck/May2015Assess.accdb',
                       'tname':'RawDataAssessment',
                       'col_remap': post_2009_colmap,
                       'dtype': post_2009_dtype
                      }
             }

In [ ]:
# Uses accdb_info map to import a file for a given year and return a dataframe.  
# This doesn't remap the column names (we might not know them yet)
def import_accdb_file(year):
    # Import the accdb file for this year.  The returned value is a map of table names to tables
    accdb_table_map = mdb_to_pandas(accdb_info[year]['fname'], accdb_info[year]['dtype'])     
    apd_df = accdb_table_map[accdb_info[year]['tname']]
    return apd_df

# Takes a dataframe imported by import_accdb_file, renames columns, sets the index to PARID, 
# filters out rows that aren't dwellings, sets zipcode to be type string
skip_dwelling_col=False
def cleanup_accdb_import(df, year):
    r_df = df.rename(index=str, columns=accdb_info[year]['col_remap']).set_index('PARID')
    if(not skip_dwelling_col):
        r_df = pd.merge(r_df,dwelling_col.to_frame(),on='PARID', left_on=None, right_on=None,
                         left_index=False, right_index=False, sort=False,
                         suffixes=('_x', '_y'), copy=True, indicator=False,
                         validate=None)
        print "After merge, size = %d" % (len(r_df))

        r_df = r_df[r_df.is_dwelling]
        print "After filtering to only include dwellings, size = %d" % (len(r_df))

    # Change ',  -' in CHANGENOTICEADDRESS3 to nan (that's a common pattern in these accdb files)
    bad_changeaddr = r_df.CHANGENOTICEADDRESS3==',  -'
    r_df.loc[bad_changeaddr,'CHANGENOTICEADDRESS3']= numpy.nan

    # Filter out empty owner names
    empty_owner = r_df.PROPERTYOWNER.isna()

    # Filter out empty change addresses
    p1=pandas.isna(r_df.CHANGENOTICEADDRESS1) 
    p2=pandas.isna(r_df.CHANGENOTICEADDRESS2) 
    p3=pandas.isna(r_df.CHANGENOTICEADDRESS3)
    p4=pandas.isna(r_df.CHANGENOTICEADDRESS4)
    r_df = r_df[~((p1&p2&p3&p4)|empty_owner)]
    print "After filtering to remove empty owner names and change addresses, size = %d" % (len(r_df))

    # Fix up <BR> to be &
    r_df.PROPERTYOWNER = r_df.PROPERTYOWNER.str.replace('\s*<BR>\s*', ' & ', regex=True)
    return r_df

# More cleanup after merge
def cleanup_merged_accdb_import(df, year):
    # A common pattern of bad addresses is ' , -0000 ', get rid of ones like this
    bad_owner_addr = df.owner_address_raw.str.contains('^[\s,]*-\d\d\d\d\s*$',regex=True)
    r_df = df[~(bad_owner_addr)]
    print "After filtering to remove malformed owner_address_raw addresses, size = %d" % (len(r_df))
    return r_df

# Process 2015

In [ ]:
apd_2015_raw = import_accdb_file('2015') 

In [ ]:
apd_2015 = cleanup_accdb_import(apd_2015_raw, '2015')

In [ ]:
merge_addrs(apd_2015)
canonicalize_addrs(apd_2015)

In [ ]:
# Save out apd_2015 to mdf
save_adf(apd_2015, 'apd_2015')

In [ ]:
process_all_assessment_records(apd_2015, '2015-05-01')

# Process 2014

In [ ]:
apd_2014_raw = import_accdb_file('2014') 

In [ ]:
apd_2014 = cleanup_accdb_import(apd_2014_raw, '2014')

In [ ]:
merge_addrs(apd_2014)
canonicalize_addrs(apd_2014)

In [ ]:
process_all_assessment_records(apd_2014, '2014-06-01')

# Process 2013

In [ ]:
apd_2013_raw = import_accdb_file('2013') 

In [ ]:
apd_2013 = cleanup_accdb_import(apd_2013_raw, '2013')

In [ ]:
merge_addrs(apd_2013)
canonicalize_addrs(apd_2013)

In [ ]:
# Save out apd_2013 to mdf
save_adf(apd_2013, 'apd_2013')

In [ ]:
process_all_assessment_records(apd_2013, '2013-12-01')

# Process 2012

In [ ]:
#  SALEDATE field is MMDDYYYY, so we need to do some more work than usual:
#    9042007  = '2007-09-04'

# Setup regular expression for parsing mmddyyyy date
mmddyyyy_re = re.compile('(\d\d)(\d\d)(\d\d\d\d)')

def mmddyyyy_to_yyyymmdd(datestr_raw):
    # Parse the sections of the date string
    m = mmddyyyy_re.match(str(datestr_raw))
    # The month will be in the first group, the day in the second, the year in the third
    return '%s-%s-%s'%(m.group(3),m.group(1),m.group(2))

In [ ]:
# If we've already processed before and haven't changed process, just reload
apd_2012 = restore_adf('apd_2012')

In [ ]:
# Otherwise, process afresh
apd_2012_raw = import_accdb_file('2012') 

In [ ]:
apd_2012 = cleanup_accdb_import(apd_2012_raw, '2012')


In [ ]:
# Get rid of empty sale dates and convert to yyyy-mm-dd format
apd_2012=apd_2012[~pandas.isna(apd_2012.SALEDATE_raw)].copy()
apd_2012['SALEDATE'] = apd_2012['SALEDATE_raw'].map(mmddyyyy_to_yyyymmdd)

In [ ]:
merge_addrs(apd_2012)
canonicalize_addrs(apd_2012)

In [ ]:
apd_2012=cleanup_merged_accdb_import(apd_2012, '2012')

In [ ]:
process_all_assessment_records(apd_2012, '2012-11-01')

# Process 2011

In [ ]:
# Restore apd_2011 from mdf if we don't want to reprocess from scratch
apd_2011 = restore_adf('apd_2011')

In [ ]:
apd_2011_raw = import_accdb_file('2011') 

In [ ]:
apd_2011 = cleanup_accdb_import(apd_2011_raw, '2011')

In [ ]:
merge_addrs(apd_2011)
canonicalize_addrs(apd_2011)

In [ ]:
# Save out apd_2011 to mdf
save_adf(apd_2011, 'apd_2011')

In [ ]:
process_all_assessment_records(apd_2011, '2011-03-01')

In [ ]:
apd_2011.loc[['0174E00236000000','0174E00234000000','0029H00025000000']]

In [ ]:
vacant_col1_2011 = (apd_2011.FAIRMARKETBUILDING<2.0)
vacant_col2_2011 = ((apd_2011['USEDESC'].fillna('').str.contains('VACANT')) | (apd_2011['USEDESC'].fillna('').str.contains('UNLOCATED PARCEL')) | (apd_2011['USEDESC'].fillna('').str.contains('CONDEMNED')))
vacant_col_2011=(vacant_col1_2011 | vacant_col2_2011)
apd_2011[vacant_col_2011]

In [ ]:
vacant_col_2011.name='is_vacant'
apd_2011 = pd.concat([apd_2011, vacant_col_2011], axis=1)

In [ ]:
df = apd_2011
fixup_cols = ['property_address_raw', 'owner_address_raw', 'property_address', 
              'owner_address','USEDESC','SALEDESC','HOMESTEADFLAG']
for col in fixup_cols:
    # Fix up to make sure there aren't differences in whitespace that could be avoided
    df[col] = df[col].str.replace('\s\s+', ' ', regex=True)
    df[col] = df[col].str.replace('^\s+', '', regex=True)
    df[col] = df[col].str.replace('\s+$', '', regex=True)

apd_2011=df

In [ ]:
# Replace zip+4 with zip
zip_p4_re = re.compile('(.*)\s(\d\d\d\d\d)-?(\d\d\d\d)?$')

def zip_p4_to_zip(zipstr):
    # Parse the sections of the zipcode
    m = zip_p4_re.match(str(zipstr))
    if(m):
        # The zip will be in the second group, the +4 in the third
        return "%s %s"%(m.group(1), m.group(2))
    else:
        return(zipstr)

apd_2011['owner_address_raw'] = apd_2011['owner_address_raw'].map(zip_p4_to_zip)


In [ ]:
# Get rid of empty sale dates
apd_2011=apd_2011[~pd.isna(apd_2011.SALEDATE)]

In [ ]:
apd_2011[apd_2011.SALEDATE.fillna('').str.contains('^[^01]', regex=True)][['SALEDATE']]

In [ ]:
apd_2011[~apd_2011.SALEDATE.str.contains('^\d\d/[0123]\d/\d\d', regex=True)][['SALEDATE']]

In [ ]:
apd_2011[apd_2011.SALEDATE.str.contains('^00/\d\d/\d\d', regex=True)][['SALEDATE']]

In [ ]:
apd_2011[apd_2011.SALEDATE.str.contains('^\d\d/00/\d\d', regex=True)][['SALEDATE']]

In [ ]:
property_map[apd_2011.iloc[0].name]

In [ ]:
property_map[apd_2011.iloc[0].name]=[{'asofdate': '2001-02-15',
      'canonical_owner_address': u'362 INDIAN RIDGE DR, CORAOPOLIS, PA, 15108',
      'changeaddr': '362 INDIAN RIDGE DR CORAOPOLIS PA 15108',
      'date': '1994-12-12',
      'event_type': 'PURCHASE',
      'fm_total': 207800,
      'is_vacant': False,
      'ownername': 'IMBROGNO PATRICK M & DENISE E (W)',
      'saleprice': 210025.0},
     {'asofdate': '2005-08-28',
      'canonical_owner_address': u'362 INDIAN RIDGE DR, CORAOPOLIS, PA, 15108',
      'changeaddr': '362 INDIAN RIDGE DR CORAOPOLIS PA 15108',
      'date': '1994-12-12',
      'event_type': 'PURCHASE',
      'fm_total': 247700,
      'is_vacant': False,
      'ownerdesc': 'REGULAR-ETUX OR ET VIR',
      'ownername': 'IMBROGNO PATRICK M & DENISE E (W)',
      'saledesc': 'VALID SALE',
      'saleprice': 210025.0},
     {'asofdate': '2009-09-01',
      'canonical_owner_address': u'362 INDIAN RIDGE DR, CORAOPOLIS, PA, 15108',
      'changeaddr': ' 362 INDIAN RIDGE DR CORAOPOLIS, PA 15108-1374 ',
      'date': u'1994-12-12',
      'event_type': 'PURCHASE',
      'fm_total': 247700.0,
      'homesteadflag': 'C',
      'is_vacant': False,
      'ownerdesc': 'Regular',
      'ownername': 'IMBROGNO PATRICK M & DENISE E (W)',
      'saledesc': 'VALID SALE',
      'saleprice': 210025.0},
     {'asofdate': '2010-06-01',
      'canonical_owner_address': u'362 INDIAN RIDGE DR, CORAOPOLIS, PA, 15108',
      'changeaddr': ' 362 INDIAN RIDGE DR CORAOPOLIS, PA 15108-1374 ',
      'date': u'1994-12-12',
      'event_type': 'PURCHASE',
      'fm_total': 247700.0,
      'homesteadflag': 'C',
      'is_vacant': False,
      'ownerdesc': 'Regular',
      'ownername': 'IMBROGNO PATRICK M & DENISE E (W)',
      'saledesc': 'VALID SALE',
      'saleprice': 210025.0},
     {'asofdate': '2017-10-01',
      'canonical_owner_address': u'362 INDIAN RIDGE DR, CORAOPOLIS, PA, 15108',
      'changeaddr': '362 INDIAN RIDGE DR CORAOPOLIS PA 15108',
      'date': u'1994-12-12',
      'event_type': 'PURCHASE',
      'fm_total': 345300,
      'homesteadflag': 'HOM',
      'is_vacant': False,
      'ownerdesc': 'REGULAR-ETUX OR ET VIR',
      'ownername': 'IMBROGNO PATRICK M & DENISE E (W)',
      'saledesc': 'VALID SALE',
      'saleprice': 210025.0},
     {'asofdate': '2018-01-22',
      'canonical_owner_address': u'362 INDIAN RIDGE DR, CORAOPOLIS, PA, 15108',
      'changeaddr': '362 INDIAN RIDGE DR CORAOPOLIS PA 15108',
      'date': u'1994-12-12',
      'event_type': 'PURCHASE',
      'fm_total': 345300,
      'homesteadflag': 'HOM',
      'is_vacant': False,
      'ownername': 'IMBROGNO PATRICK M & DENISE E (W)',
      'saledesc': 'VALID SALE',
      'saleprice': 210025.0,
      'usedesc': 'SINGLE FAMILY'}]


In [ ]:
process_assessment_record(apd_2011, 0, '2011-03-01')

# Process 2010

In [ ]:
# Restore apd_2010 from mdf if we don't want to reprocess from scratch
apd_2010 = restore_adf('apd_2010')

In [ ]:
apd_2010_raw = import_accdb_file('2010') 

In [ ]:
apd_2010 = cleanup_accdb_import(apd_2010_raw, '2010')

In [ ]:
merge_addrs(apd_2010)
canonicalize_addrs(apd_2010)

In [ ]:
# Alternatively, load apd_2010 from storage
apd_2010 = restore_adf('apd_2010')

In [ ]:
# Try to save out property_map before we start messsing with it
import pickle

with open('assessments/property_map_99_04_05_09_17.pickle', 'wb') as handle:
    pickle.dump(property_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('assessments/owner_map_99_04_05_09_17.pickle', 'wb') as handle:
    pickle.dump(owner_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('assessments/changeaddr_map_99_04_05_09_17.pickle', 'wb') as handle:
    pickle.dump(changeaddr_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('assessments/canonical_property_address_map_99_04_05_09_17.pickle', 'wb') as handle:
    pickle.dump(canonical_property_address_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('assessments/raw_property_address_map_99_04_05_09_17.pickle', 'wb') as handle:
    pickle.dump(raw_property_address_map, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# Save out apd_2010 to mdf
save_adf(apd_2010, 'apd_2010')

In [ ]:
process_all_assessment_records(apd_2010, '2010-06-01')

In [ ]:
# The following are historical fixups I had to do for the 2010 data set, but which shouldn't need to be done again

In [ ]:
col_rename_2010_augment = {'FairMarketLand':'FAIRMARKETLAND',
'FairMarketBuilding':'FAIRMARKETBUILDING'}
apd_2010= apd_2010.rename(index=str, columns=col_rename_2010_augment)

In [ ]:
apd_2010.loc[['0174E00236000000','0174E00234000000','0029H00025000000']]

In [ ]:
vacant_col1_2010 = (apd_2010.FAIRMARKETBUILDING<2.0)
vacant_col2_2010 = ((apd_2010['USEDESC'].fillna('').str.contains('VACANT')) | (apd_2010['USEDESC'].fillna('').str.contains('UNLOCATED PARCEL')) | (apd_2010['USEDESC'].fillna('').str.contains('CONDEMNED')))
vacant_col_2010=(vacant_col1_2010 | vacant_col2_2010)
apd_2010[vacant_col_2010]

In [ ]:
vacant_col_2010.name='is_vacant'
apd_2010 = pd.concat([apd_2010, vacant_col_2010], axis=1)

# Process 2009

In [ ]:
# Restore apd_2009 from mdf if we don't want to reprocess from scratch
apd_2009 = restore_adf('apd_2009')

In [ ]:
apd_2009_raw = import_accdb_file('2009') 

In [ ]:
apd_2009 = cleanup_accdb_import(apd_2009_raw, '2009')

In [ ]:
merge_addrs(apd_2009)

In [ ]:
apd_2009=cleanup_merged_accdb_import(apd_2009, '2009')


In [ ]:
canonicalize_addrs(apd_2009)

In [ ]:
process_all_assessment_records(apd_2009, '2009-09-01')

In [ ]:
len(apd_2009)

In [ ]:
# Save out apd_2009 to mdf
save_adf(apd_2009, 'apd_2009')

In [ ]:
# The following are historical fixups I had to do for the 2009 data set, but which shouldn't need to be done again

In [ ]:
col_rename_2009_augment = {'FAIRMARK_1':'FAIRMARKETLAND',
'FAIRMARKET':'FAIRMARKETBUILDING'}
apd_2009= apd_2009.rename(index=str, columns=col_rename_2009_augment).copy()

In [ ]:
apd_2009.loc[['0174E00236000000','0174E00234000000','0029H00025000000']]

In [ ]:
vacant_col1_2009 = (apd_2009.FAIRMARKETBUILDING<2.0)
vacant_col2_2009 = ((apd_2009['USEDESC'].fillna('').str.contains('VACANT')) | (apd_2009['USEDESC'].fillna('').str.contains('UNLOCATED PARCEL')) | (apd_2009['USEDESC'].fillna('').str.contains('CONDEMNED')))
vacant_col_2009=(vacant_col1_2009 | vacant_col2_2009)
apd_2009[vacant_col_2009]

In [ ]:
vacant_col_2009.name='is_vacant'
apd_2009 = pd.concat([apd_2009, vacant_col_2009], axis=1)

# After input processing is done, register all sales so we know when owners stop owning given parcels

In [ ]:
start=arrow.now()
register_all_sales()
end=arrow.now()
print "Processing took %s" % (str(end-start))

# Debugging

In [ ]:
len(property_map)

In [ ]:
len(owner_map)

In [ ]:
property_map['9946X83377000000']

In [ ]:
owner_map['ANDREW ALAN S & SUSAN B (W)']

In [ ]:
len(changeaddr_map['PO BOX 8469 CANTON OH 44711'])

In [ ]:
property_map['0029C00050000000']

In [ ]:
property_map['2018H00310000000']

In [ ]:
property_map['0028H00014000000']=[{'changeaddr': '3535 BLVD OF THE ALLIES, PITTSBURGH, PA 15213',
  'date': u'1986-08-25',
  'event_type': 'PURCHASE',
  'ownername': 'KELLY ROBERT E JR & KELLY ROBERT E JR WILLIAM W RIELLY ROBERT B NELL JR & HOMER E STOTLER'},
 {'changeaddr': '3535 BLVD OF THE ALLIES, PITTSBURGH, PA 15213',
  'date': u'1986-08-25',
  'event_type': 'PURCHASE',
  'ownername': 'KELLY ROBERT E JR & RIELLY WILLIAM W & NELL ROBE'},
 {'changeaddr': '3535 BLVD OF THE ALLIES, PITTSBURGH, PA 15213',
  'date': u'2012-12-18',
  'event_type': 'PURCHASE',
  'ownername': 'ROBERT E KELLY JR REVOCABLE TRUST'},
 {'changeaddr': '3535 BLVD OF THE ALLIES, PITTSBURGH, PA 15213',
  'date': u'2012-12-18',
  'event_type': 'PURCHASE',
  'ownername': 'ROBERT E KELLY JR REVOCABLE TRUST & WILLIAM W RIELLY REVOCABLE TRU'},
 {'changeaddr': '3535 BLVD OF THE ALLIES, PITTSBURGH, PA 15213',
  'date': u'2012-12-18',
  'event_type': 'PURCHASE',
  'ownername': 'ROBERT E KELLY JR REVOCABLE TRUST ROBERT E KELLY JR TRUSTEE & WIL'}]

In [ ]:
owner_map['ROBERT E KELLY JR REVOCABLE TRUST ROBERT E KELLY JR TRUSTEE & WIL']=owner_map['ROBERT E KELLY JR REVOCABLE TRUST ROBERT E KELLY JR \xce\x93\xc3\x87\xc3\xb4 TRUSTEE & WIL']

In [ ]:
changeaddr_map['8 WATSON ST, CARNEGIE, PA 15106']

In [ ]:
owner_map['STANDARD REALTY GROUP LP']

In [ ]:
# Sanity check it's working
next_date = '1970-01-01'
parid='0084J00135000000'
print property_map[parid]
while(next_date<this_date):
    ret=get_related_parids(parid, next_date)
    print "%s: %s - %s = %d" % (parid, ret['start_date'], ret['next_date'], ret['parcount'])
    next_date=ret['next_date']

In [ ]:
# TORKEO ROBERT V
next_date = '1970-01-01'
parid='0029H00175000000'
print property_map[parid]
while(next_date<this_date):
    ret=get_related_parids(parid, next_date)
    print "%s: %s - %s = %d" % (parid, ret['start_date'], ret['next_date'], ret['parcount'])
    next_date=ret['next_date']

In [ ]:
print property_map.keys()[1]
property_map[property_map.keys()[1]]

In [ ]:
apd_2009[apd_2009.PROPERTYOWNER.str.contains('WYDRENSKI ')]

In [ ]:
apd_2009.loc['0163B00066000000']

In [ ]:
apd_2017.loc['0162D00020000000']

In [ ]:
apd_2009[~apd_2009.index.str.contains('000000$',regex=True)]

In [ ]:
apd_2009

# Process geometries to be able to record lat/lon for properties

In [ ]:
#with open("assessments/Allegheny_County_Parcel_Boundaries.geojson") as f:
#    parcel_json = json.load(f)
#len(parcel_json['features'])
g = gpd.read_file('assessments/Allegheny_County_Parcel_Boundaries.geojson')

g=g.set_index('PIN')

# Calculate the centroid for each row and store in a new column called centroid
g['centroid']=g['geometry'].centroid

In [ ]:
# Create a map from PARID to centroid
parid2centroid = {g.index[i]:g['centroid'].iloc[i] for i in range(0, len(g.index))}

In [ ]:
# Given a list of parcel ids, add a centroid to parid2centroid from the property address if we have a good one
def update_centroids(parid_arr):
    global canonical_property_address_map
    global raw_property_address_map
    for parid in parid_arr:
        # Check if we already know this centroid
        if(parid in parid2centroid):
            continue
        # Don't know the centroid yet.  Look it up
        if parid in canonical_property_address_map:
            addr=canonical_property_address_map[parid]
            coords = get_canonical_coords(addr)
            if(coords):
                print "Updating %s from stored canonical addr: %r (%f, %f)" % (parid, addr, coords.x, coords.y)
                parid2centroid[parid]=coords
                continue
        if parid in raw_property_address_map:
            raw_addr = raw_property_address_map[parid]
            cp_addr = update_canonical_address_from_raw(None, raw_addr, False)
            # Cache for next time
            canonical_property_address_map[parid]=cp_addr
            coords = get_canonical_coords(cp_addr)
            if(coords):
                print "Updating %s from raw: %r -> %r (%f, %f)" % (parid, raw_addr, cp_addr, coords.x, coords.y)
                parid2centroid[parid]=coords
                continue
            print "Skipping %s: Raw property addr = %r, %r" % (parid, raw_addr, cp_addr)
        else:
            print "Skipping %s: No raw property addr" % (parid)


In [ ]:
# Do fixup on centroids
redo_propaddr_parids=['0023B00228000000','0083R00164000000']

In [ ]:
raw_property_address_map['0023B00228000000']='1641 PERRYSVILLE AVE PITTSBURGH PA 15212'
raw_property_address_map['0083R00164000000']='340 N SHERIDAN AVE PITTSBURGH PA 15206'
for parid in redo_propaddr_parids:
    ca = update_canonical_address_from_raw(None, raw_property_address_map[parid], False)
    if(not canonical_addresses_empty(ca)):
        canonical_property_address_map[parid]=ca
    elif(parid in canonical_property_address_map):
        del canonical_property_address_map[parid]

In [ ]:
for parid in redo_propaddr_parids:
    print "%s: %r" % (parid,canonical_property_address_map[parid] if parid in canonical_property_address_map else None)

In [ ]:
update_centroids(redo_propaddr_parids)

In [ ]:
bad_centroid_parids = [line.rstrip('\n') for line in open('assessments/ownerprox_99_04_05_09_17_missing.txt')]

In [ ]:
for parid in bad_centroid_parids:
    print "%s: %r" % (parid,canonical_property_address_map[parid] if parid in canonical_property_address_map else None)

In [ ]:
update_centroids(bad_centroid_parids)

# New volume animation

In [ ]:
def color_from_floats(r,g,b):
    return r + g * 256.0 + b * 256.0 * 256.0

def pack_color(color):
    return color['r'] + color['g'] * 256.0 + color['b'] * 256.0 * 256.0;

def parse_color(color):
    color = color.strip()
    c = color
    try:
        if c[0] == '#':
            c = c[1:]
        if len(c) == 3:
            return pack_color({'r': 17 * int(c[0:1], 16),
                               'g': 17 * int(c[1:2], 16),
                               'b': 17 * int(c[2:3], 16)})
        if len(c) == 6:
            return pack_color({'r': int(c[0:2], 16),
                               'g': int(c[2:4], 16),
                               'b': int(c[4:6], 16)})
    except:
        pass
    raise InvalidUsage('Cannot parse color <code><b>%s</b></code> from spreadsheet.<br><br>Color must be in standard web form, <code><b>#RRGGBB</b></code>, where RR, GG, and BB are each two-digit hexadecimal numbers between 00 and FF.<br><br>See <a href="https://www.w3schools.com/colors/colors_picker.asp">HTML Color Picker</a>' % color)

def parse_colors(colors):
    packed = [parse_color(color) for color in colors]
    return numpy.array(packed, dtype = numpy.float32)

In [ ]:
#out_suffix="_09_10_11_13_14_15_17"
#out_suffix="_15213_17"
#out_suffix="_99_09_17"
#out_suffix="_99_04b_09_17"
#out_suffix="_99b_04_05_09_17"
#out_suffix="_99_04_05_09_10_17"
#out_suffix="99_04c_05_09_10_17"
out_suffix="99b_04d_05_09_10_11_17_18b"

In [ ]:
start=arrow.now()

# Use the property_map, owner_map, and changeaddr_map to generate a new type of volume animation
# Write out a binary file with the volume colors
vol_colors = ['#a50026','#cd2827','#e75436','#f7804b','#fdad61','#fed788','#ffffbf','#b9e0ed','#8dc0db','#699fca','#4d7db9','#3e5aa7','#313695']
def volume_to_color(volume):
    if (volume < 2):
        return parse_color(vol_colors[0]) 
    #elif (volume < 3):
    #    return parse_color(vol_colors[1]) 
    #elif (volume < 4):
    #    return parse_color(vol_colors[2]) 
    elif (volume < 5):
        return parse_color(vol_colors[3]) 
    elif (volume < 10):
        return parse_color(vol_colors[4]) 
    elif (volume < 20):
        return parse_color(vol_colors[5]) 
    elif (volume < 40):
        return parse_color(vol_colors[6]) 
    elif (volume < 60):
        return parse_color(vol_colors[7]) 
    elif (volume < 80):
        return parse_color(vol_colors[8]) 
    elif (volume < 150):
        return parse_color(vol_colors[9]) 
    elif (volume < 300):
        return parse_color(vol_colors[10]) 
    elif (volume < 500):
        return parse_color(vol_colors[11]) 
    else:
        return parse_color(vol_colors[12]) 

def output_volume_dots():
    # Write out volume of ownership for each residential non-vacant land property
    points = []
    i=0
    start=arrow.now()
    chunk_start_time=arrow.now()
    chunk_size=1000

    for parid in property_map.keys():
        centroid=None
        try:
            centroid = parid2centroid[parid]
        except:
            print "%s is missing from centroids, skipping" % (parid)
            continue

        try:
            # Handle periodic debug message
            if((i%chunk_size)==0 and i>0):
                print "%d-%d: processing %r, %s time elapsed" %(i-(chunk_size-1), i, parid, arrow.now()-chunk_start_time)
                addcnt=0
                chunk_start_time=arrow.now()

            # Nominally start at 1950
            next_date = '1950-01-01'

            while(next_date<this_date):
                volume_map=get_related_parids(parid, next_date)

                next_date=volume_map['next_date']
                color = volume_to_color(volume_map['parcount'])
                if((i%chunk_size)==0):
                    print "  %s: %s - %s = %d" % (parid, volume_map['start_date'], volume_map['next_date'], volume_map['parcount'])
                saledate = SaledateToEpoch(volume_map['start_date'])
                enddate = SaledateToEpoch(next_date)
                if(color != None):
                    points += PointToPixelXY(centroid)     
                    points.append(color)
                    # Put epoch time for SALEDATE as start valid time, and next_date as end valid time
                    points.append(float(saledate))
                    points.append(float(enddate))
                else:
                    print "Color of " + str(volume_map['parcount']) + " is None"
        except:
            print "Unexpected error processing %s, next_date=%s:" % (parid, next_date), sys.exc_info()[0]
            #raise

        #Increment debug message counter
        i=i+1

    array.array('f', points).tofile(open(('assessments/res_volume_color_m_epoch%s.bin'%out_suffix), 'wb'))

    end=arrow.now()
    print "Processing took %s" % (str(end-start))
    
#import cProfile
#cProfile.run('output_volume_dots()')
output_volume_dots()


In [ ]:
# Call this version if you're ok with potentially extra computation 
# It checks if there's a census entry. If not, it generates one.
# If that fails, it checks if there's a google entry.  If not, it generates one.
# it returns the resulting map
def update_canonical_address_from_raw(ca_in, raw_addr, force_google):
    # Check if we have a valid ca_in, if not create with census from raw_addr
    if(pd.isnull(ca_in) or ('census' not in ca_in)):
       # We don't have anything yet, call census on raw_addr
       ca_in = {'census':get_canonical_address(raw_addr)}
    # Check for value of census field
    if(not force_google and ca_in['census']):
       # Census is good, not forcing computation of google, stop there
       return ca_in
    # No census canonicalization, or force_google is True, check for value in google field
    if('google' not in ca_in):
        ca_in['google']=get_canonical_address_google(raw_addr)
        print "Checked google for %r: %r" % (raw_addr, ca_in)
    return(ca_in)

# This version doesn't try to go deeper.  Call this if you don't want more computation to happen
def canonical_addresses_match(ca1,ca2):
    # Check if both are non-null
    #print "Comparing %r and %r" % (ca1, ca2)
    if(pd.isnull(ca1) or pd.isnull(ca2)):
        # We don't know all we could know here in case there's a raw address available
        #print "   Incomplete: %r (%r vs %r)" % (None, ca1, ca2)
        return None
    # Check if they both have census entries
    if('census' in ca1 and 'census' in ca2 and ca1['census'] and ca2['census']):
        ret= ca1['census']==ca2['census']
        #print "   Census: %r" % (ret)
        return (ret)
    # One or both are missing census, try google
    if('google' in ca1 and 'google' in ca2):
        # Both have been computed, return false if either is empty or both are non-empty and they don't match
        if(ca1['google'] and ca1['google']):
            ret = ca1['google']==ca2['google']
            #print "   Google: %r" % (ret)
            return (ret)
        else:
            #print "   Mismatch: %r" % (False)
            return False
    # Can't determine equality, more updating might be needed return None
    #print "   Incomplete: %r (%r vs %r)" % (None, ca1, ca2)
    return None
       
# Returns True if both census and google have been tried and failed
def canonical_addresses_empty(ca):
    if(pd.isnull(ca) or (('census' in ca) and pd.isnull(ca['census'])) and (('google' in ca) and (pd.isnull(ca['google'])))):
        return True
    elif(('census' in ca) and not pd.isnull(ca['census'])):
        return False
    elif(('google' in ca) and not pd.isnull(ca['google'])):
        return False
    # More could be done here, return Null
    return None                                                               

In [ ]:
start=arrow.now()

single_property_color = '#a50026'
corporate_color = '#ffffbf'
unknown_color = '#303030'

def ownertype_to_color(parid, property_event):
    # Check if property and owner canonical addresses are both known
    property_canonical_addr=get_canonical_property_addr(parid)
    owner_canonical_addr=None
    if('canonical_owner_address' in property_event.keys()):
        owner_canonical_addr=property_event['canonical_owner_address']
    # Check if both addresses are identical.  If so, use single_property_color
    # since it's owner occupied, even if the owner is a multiple or corporate owner
    # in general
    if(property_canonical_addr and owner_canonical_addr and canonical_addresses_match(property_canonical_addr,owner_canonical_addr)):
        return single_property_color

    ownerdesc = 'REGULAR'
    if('ownerdesc' in property_event.keys()):
        ownerdesc = property_event['ownerdesc']
    volume_map=get_related_parids(parid, property_event['date'])
    volume = volume_map['parcount']
    if ('CORPORATION' in ownerdesc or 'Corporation' in ownerdesc):
        # Ivory 
        return corporate_color
    elif ('homesteadflag' in property_event.keys() and 
          (property_event['homesteadflag'].strip()=='HOM' or property_event['homesteadflag'].strip()=='C')):
        # If homestead flag set, use single property color regardless of volume
        return single_property_color
    elif ('REGULAR' in ownerdesc or 'Regular' in ownerdesc):
        # Regular owner, what volume?
        if(volume == 1):
            # Same color as 1 in volume view
            return single_property_color
        else:
            # Same color as other in class view
            return '#02ca75'
    else:
        print "Unrecognized owner type: %s, %d" % (ownerdesc,volume)
        return unknown_color

def output_ownertype_dots(parid_arr, suffix):
    # If parid_arr not specified, do all of the keys in property_map
    if(len(parid_arr)==0):
        parid_arr=property_map.keys()
        
    # Write out volume of ownership for each residential non-vacant land property
    points = []
    start=arrow.now()
    chunk_start_time=arrow.now()
    chunk_size=1000
    chunk_cnt=0
    
    for parid in parid_arr:
        centroid=None
        did_output_dot=False
        try:
            centroid = parid2centroid[parid]
        except:
            print "%s is missing from centroids, skipping" % (parid)
            continue

        try:
            # Handle periodic debug message
            if((chunk_cnt%chunk_size)==0 and chunk_cnt>0):
                print "%d-%d: processing %r, %s time elapsed" %(chunk_cnt-(chunk_size-1), chunk_cnt, parid, arrow.now()-chunk_start_time)
                addcnt=0
                chunk_start_time=arrow.now()

            # Get list of property events or this property
            property_events = property_map[parid]

            # Keep track of the date of the last datapoint. 
            last_date = '1900-01-01'
            last_color=0
            
            for i in range(0,len(property_events)):
                event_date = property_events[i]['date']
                # Get color for this property event
                color = ownertype_to_color(parid, property_events[i])
                # Check if the date and color haven't changed, if so skip to the next event
                if(color == last_color and last_date == event_date):
                    # Nothing new here, move along
                    continue
                elif(last_date==event_date and i>0):
                    # Color changed without the sale date changing, possibly flag this as an issue
                    # If this new color is unknown, skip it
                    # If the last color was unknown, delete it and use this one
                    # Otherwise, this might be a change in homesteadflag.  If so, single_property_color takes precedence.
                    # Corporate takes priority between multi and corporate
                    # If neither has single_property_color, raise an exception
                    if(color==unknown_color):
                        # Ok, just skip this one
                        continue
                    elif(last_color==unknown_color):
                        # Pop the previous point off (5 floats) and use this new one
                        for j in range(0,5):
                            del points[-1]
                        #print "  REMOVED %s: %s = %s (len %d)" % (parid, event_date,last_color, len(points))
                    elif(last_color==single_property_color):
                        # Ok, just skip this one
                        continue
                    elif(color == single_property_color):
                        # Pop the previous point off (5 floats) and use this new one
                        for j in range(0,5):
                            del points[-1]
                        #print "  REMOVED %s: %s = %s (len %d)" % (parid, event_date,last_color, len(points))
                    elif(last_color==corporate_color):
                        # Ok, just skip this one
                        continue
                    elif(color==corporate_color):
                        # Pop the previous point off (5 floats) and use this new one
                        for j in range(0,5):
                            del points[-1]
                        #print "  REMOVED %s: %s = %s (len %d)" % (parid, event_date,last_color, len(points))
                    else:
                        print " PROBLEM %s: %s changed color on same date %s->%s" % (parid, event_date, last_color, color)
                        raise
                        
                if((chunk_cnt%chunk_size)==0):
                    print "  %s: %s = %s" % (parid, event_date, color)
                
                # Get range of dates for this color
                saledate = SaledateToEpoch(event_date)
                next_date = get_next_sale_date(parid, event_date)
                if(next_date == None):
                    # No new owner after this, set end date to the end of time
                    enddate = float(1e38)
                    next_date=this_date
                else:
                    enddate = SaledateToEpoch(next_date)
                if(color != None):
                    points += PointToPixelXY(centroid)     
                    points.append(parse_color(color))
                    # Put epoch time for SALEDATE as start valid time, and next_date as end valid time
                    points.append(float(saledate))
                    points.append(float(enddate))
                    #print "  %s: %s - %s = %s (len %d)" % (parid, event_date, next_date, color, len(points))
                else:
                    print "Color of " + str(volume_map['parcount']) + " is None"

                # Set last_date and last_color for next loop
                last_date = event_date
                last_color= color
                did_output_dot=True
        except:
            print "Unexpected error processing %s:" % (parid), sys.exc_info()[0]
            raise

        #Increment debug message counter
        if(did_output_dot):
            chunk_cnt = chunk_cnt+1

    array.array('f', points).tofile(open(('assessments/ownertype_color_m_epoch%s.bin'%suffix), 'wb'))

    end=arrow.now()
    print "Processing took %s for %d dots" % (str(end-start), chunk_cnt)
    

In [ ]:
#import cProfile
#cProfile.run('output_volume_dots()')
output_ownertype_dots([],out_suffix)

In [ ]:
parids_15213=apd_2017[apd_2017.PROPERTYZIP=='15213'].index

In [ ]:
valid_parids_15213=list(set(parids_15213) & set(property_map.keys()) & set(parid2centroid.keys()))

In [ ]:
len(valid_parids_15213)

In [ ]:
valid_parids_15217=list(set(apd_15217_2017.index) & set(property_map.keys()))

In [ ]:
len(valid_parids_15217)

In [ ]:
output_ownertype_dots(valid_parids_15213,"_d4_15213")

In [ ]:
start=arrow.now()

single_property_color = '#a50026'
corporate_color = '#ffffbf'
unknown_color = '#303030'

# Distressed sales are:
distressed_saledescs=['SHERIFF SALE', 'CITY TREASURER SALE','CITY TREASURER',
                      'PREV FORECLOSE','GOVT SALE','BANK SALE']
#affection_saledescs=['LOVE AND AFFECTION SALE','LOVE&AFFECTION']
affection_saledescs=[]

#distressed_colors=["#ff0000","#00ff00"]
#affection_colors=["#FA00D4","#15FA00"]
distressed_colors=["#FA00D4","#15FA00"]
affection_colors=[]

single_property_colors=[single_property_color, "#ff0000", "#FA00D4"]
def saletype_to_color(parid, property_event):
    try:
        ownerdesc = 'REGULAR'
        if('ownerdesc' in property_event.keys()):
            ownerdesc = property_event['ownerdesc']
        saledesc = 'VALID'
        if('saledesc' in property_event.keys()):
            saledesc = property_event['saledesc']
        volume_map=get_related_parids(parid, property_event['date'])
        volume = volume_map['parcount']

        # Check if homestead flag is set
        homesteadflag=False
        if('homesteadflag' in property_event.keys() and 
          (property_event['homesteadflag'].strip()=='HOM' or property_event['homesteadflag'].strip()=='C')):
            homesteadflag=True
        # Check if property went to a single owner or not
        single_owner = True
        if ('CORPORATION' in ownerdesc or 'Corporation' in ownerdesc or (volume>1 and not homesteadflag)):
            single_owner=False
        
        
        # Check for distressed sale
        if(saledesc in distressed_saledescs):
            # Is distressed sale, check if it went to a single owner or not
            if(single_owner):
                # Set to bright pink
                return "#FA00D4"
            else:
                # Return fluorescent green
                return "#15FA00"
#            if(single_owner):
#                # Set to bright red
#                return "#ff0000"
#            else:
#                # Return bright green
#                return "#00ff00"
#        elif(saledesc in affection_saledescs):
#            # Is affection sale, check if it went to a single owner or not
#            if(single_owner):
#                # Set to bright pink
#                return "#FA00D4"
#            else:
#                # Return fluorescent green
#                return "#15FA00"
        else:
            # Normal sale
#            if ('CORPORATION' in ownerdesc or 'Corporation' in ownerdesc):
#                # Ivory 
#                return '#ffffbf'
            if (single_owner):
                # Same color as 1 in volume view
                return single_property_color
            else:
                # Non-corporate multi-owner, reguarl sale
                # Same color as other in class view
                return '#02ca75'
    except:
        print "Exception processing saletype: %s" % (parid), sys.exc_info()[0]
        raise
        return '#303030'

def output_saletype_dots(parid_arr, suffix):
    # If parid_arr not specified, do all of the keys in property_map
    if(len(parid_arr)==0):
        parid_arr=property_map.keys()
    # Write out volume of ownership for each residential non-vacant land property
    points = []
    start=arrow.now()
    chunk_start_time=arrow.now()
    chunk_size=1000
    chunk_cnt=0
    
    for parid in parid_arr:
        centroid=None
        did_output_dot=False
        try:
            centroid = parid2centroid[parid]
        except:
            print "%s is missing from centroids, skipping" % (parid)
            continue

        try:
            # Handle periodic debug message
            if((chunk_cnt%chunk_size)==0 and chunk_cnt>0):
                print "%d-%d: processing %r, %s time elapsed" %(chunk_cnt-(chunk_size-1), chunk_cnt, parid, arrow.now()-chunk_start_time)
                addcnt=0
                chunk_start_time=arrow.now()

            # Get list of property events or this property
            property_events = property_map[parid]

            # Keep track of the date of the last datapoint. 
            last_date = '1900-01-01'
            last_color=0
            
            for i in range(0,len(property_events)):
                event_date = property_events[i]['date']
                # Get color for this property event
                color = saletype_to_color(parid, property_events[i])
                # Check if the date and color haven't changed, if so skip to the next event
                if(color == last_color and last_date == event_date):
                    # Nothing new here, move along
                    continue
                elif(last_date==event_date and i>0):
                    # Color changed without the sale date changing, possibly flag this as an issue
                    # If this new color is unknown, skip it
                    # If the last color was unknown, delete it and use this one
                    # Otherwise, distressed_colors take precedence if present, 
                    # if not, this might be a change in homesteadflag.  If so, single_property_color takes precedence.
                    # Corporate takes priority between multi and corporate
                    # If neither has single_property_color, raise an exception
                    if(color==unknown_color):
                        # Ok, just skip this one
                        continue
                    elif(last_color==unknown_color):
                        # Pop the previous point off (5 floats) and use this new one
                        for j in range(0,5):
                            del points[-1]
                    elif(last_color in distressed_colors):
                        # Ok, just skip this one
                        continue
                    elif(color in distressed_colors):
                        # Pop the previous point off (5 floats) and use this new one
                        for j in range(0,5):
                            del points[-1]
                    elif(last_color in affection_colors):
                        # Ok, just skip this one
                        continue
                    elif(color in affection_colors):
                        # Pop the previous point off (5 floats) and use this new one
                        for j in range(0,5):
                            del points[-1]
                    elif(color in single_property_colors):
                        # Pop the previous point off (5 floats) and use this new one
                        for j in range(0,5):
                            del points[-1]
                    elif(last_color in single_property_colors):
                        # Ok, just skip this one
                        continue
                    elif(last_color==corporate_color):
                        # Ok, just skip this one
                        continue
                    elif(color==corporate_color):
                        # Pop the previous point off (5 floats) and use this new one
                        for j in range(0,5):
                            del points[-1]
                    else:
                        print " PROBLEM %s: %s changed color on same date %s->%s" % (parid, event_date, last_color, color)
                        raise

                if((chunk_cnt%chunk_size)==0):
                    print "  %s: %s = %s" % (parid, event_date, color)
                
                # Get range of dates for this color
                saledate = SaledateToEpoch(event_date)
                next_date = get_next_sale_date(parid, event_date)
                if(next_date == None):
                    # No new owner after this, set end date to the end of time
                    enddate = float(1e38)
                    next_date=this_date
                else:
                    enddate = SaledateToEpoch(next_date)
                if(color != None):
                    points += PointToPixelXY(centroid)     
                    points.append(parse_color(color))
                    # Put epoch time for SALEDATE as start valid time, and next_date as end valid time
                    points.append(float(saledate))
                    points.append(float(enddate))
                    #print "  %s: %s - %s = %s (len %d)" % (parid, event_date, next_date, color, len(points))
                else:
                    print "Color of " + str(volume_map['parcount']) + " is None"

                # Set last_date and last_color for next loop
                last_date = event_date
                last_color= color
                did_output_dot=True
        except:
            print "Unexpected error processing %s:" % (parid), sys.exc_info()[0]
            raise

        #Increment debug message counter
        if(did_output_dot):
            chunk_cnt = chunk_cnt+1

    array.array('f', points).tofile(open(('assessments/saletype_color_m_epoch%s.bin'%suffix), 'wb'))

    end=arrow.now()
    print "Processing took %s for %d dots" % (str(end-start), chunk_cnt)
    

In [ ]:
output_saletype_dots([],out_suffix)

In [ ]:
parid='0121K00245000000'
print canonical_property_address_map[parid]
print property_map[parid]
for j in range(0,len(property_map[parid])):
    print '%r: %s\n' % (property_map[parid][j], ownerprox_to_color(parid, property_map[parid][j]))

In [ ]:
'#a50026'  in single_property_colors

In [ ]:
output_saletype_dots(['0028L00264000000'],"_d1")

In [ ]:
output_saletype_dots(valid_parids_15213,"_15213")

# Owner proximity

In [ ]:
# Create a cache of zip code to proximity. 
# If zip is in PA, but not Allegheny county, value will be 'PA'
# If zip is in Allegheny county, value will be 'Allegheny'
zip_map={}

In [ ]:
# Read in US zips, subset on PA
us_zips = pd.read_csv("allegheny_county/zip/us-national-zips-2010.csv",dtype={'zipcode':numpy.str})
pa_zips = us_zips[us_zips.state=='PA']

In [ ]:
for zipcode in pa_zips['zipcode']:
    zip_map[zipcode]='PA'

In [ ]:
# List of Allegheny county zipcodes adapted from https://clicknathan.com/2012/10/10/csv-of-all-zip-codes-in-allegheny-county-and-pittsburgh/
ac_zips = [15006,15007,15101, 15006,15102,15014,15104,15015,15017,15106,15024,15025,15026,15108,15030,15046,15034,15110,15035,15112,15037,15044,15045,15116,15120,15126,15056,15129,15136,15130,15131,15132,15133,15135,15146,15064,15065,15137,15071,15139,15140,15201,15202,15203,15204,15205,15206,15207,15208,15209,15210,15211,15212,15213,15214,15215,15216,15217,15218,15219,15220,15221,15222,15223,15224,15225,15226,15227,15228,15229,15232,15233,15234,15235,15236,15237,15238,15239,15241,15242,15243,15275,15276,15142,15076,15143,15144,15084,15145,15147,15086,15122,15123,15090,15148]
for zipcode in ac_zips:
    zip_map[str(zipcode)]='Allegheny'

In [ ]:
property_map['0029G00190000000']

In [ ]:
parid = '0062G00184000000'
sevs = get_simultaneous_property_events(property_map[parid],0)
colors = map(lambda ev: ownerprox_to_color(parid, ev), sevs)
colors

In [ ]:
# Use a set for accumulating parids for ownerprox to skip since we don't want duplicates
# This is an optimization.  Re-execute if new entries are added to parid2centroid or property address related maps
ownerprox_skip_parids=set()

In [ ]:
start=arrow.now()

vol_colors = ['#a50026','#cd2827','#e75436','#f7804b','#fdad61','#fed788','#ffffbf','#b9e0ed','#8dc0db','#699fca','#4d7db9','#3e5aa7','#313695']

sameaddr_color = vol_colors[0]
samezip_color = vol_colors[3]
allegheny_co_color=vol_colors[6]
in_state_color=vol_colors[8]
out_of_state_color=vol_colors[11]

unknown_color = '#303030'

def ownerprox_to_color(parid, property_event):
    global canonical_property_address_map
    global raw_property_address_map
    global property_map

    # Record which sort of canonical address we have, as they can't be
    # compared against each other
    use_census_addrs=True
    
    # Check if property and owner canonical addresses are both known
    # to census
    property_canonical_addr=get_canonical_property_addr(parid)
    owner_canonical_addr=None
    if('canonical_owner_address' in property_event.keys()):
        owner_canonical_addr=property_event['canonical_owner_address']
    # if we already know enough to compare them.  If so, this returns True or False.
    # If not, it returns None and we potentially need to do updating from raw
    #print "  %r: Comparing (%r, %r)" % (parid, property_canonical_addr, owner_canonical_addr)
    addrs_match = canonical_addresses_match(property_canonical_addr,owner_canonical_addr)
    if(pd.isnull(addrs_match)):
        # Don't know enough to answer, potentially update both
        #print "  %r: Canonical address comparison requires more processing (%r, %r)" % (parid, property_canonical_addr, owner_canonical_addr)
        # Missing something, try census first
        property_canonical_addr = update_canonical_address_from_raw(property_canonical_addr, raw_property_address_map[parid], False)
        owner_canonical_addr = update_canonical_address_from_raw(owner_canonical_addr, property_event['changeaddr'], False)
        
        # Check if it was enough to not force google processing
        addrs_match = canonical_addresses_match(property_canonical_addr,owner_canonical_addr)
        if(pd.isnull(addrs_match)):
            print "  %r: Forcing google address processing(%r, %r)" % (parid, property_canonical_addr, owner_canonical_addr)
            property_canonical_addr = update_canonical_address_from_raw(property_canonical_addr, raw_property_address_map[parid], True)
            owner_canonical_addr = update_canonical_address_from_raw(owner_canonical_addr, property_event['changeaddr'], True)
            # Check one more time after forcing google processing
            addrs_match = canonical_addresses_match(property_canonical_addr,owner_canonical_addr)
        
        # Cache result for next time
        canonical_property_address_map[parid] = property_canonical_addr
        # This is messy bcause property_event is a copy and we need to modify the global version
        pm_update_index = find_event_index_by_date_and_asofdate(property_map[parid], property_event['date'], property_event['asofdate'])
        if(pm_update_index>=0 and property_map[parid][pm_update_index]['date']==property_event['date'] and property_map[parid][pm_update_index]['asofdate']==property_event['asofdate']):
            property_map[parid][pm_update_index]['canonical_owner_address'] = owner_canonical_addr
    
        if(pd.isnull(addrs_match)):
            print "  %r: addrs_match is None after forcing google address processing(%r, %r)" % (parid, property_canonical_addr, owner_canonical_addr)
            raise
        
    # If property_canonical_addr is empty add parid to ownerprox_skip_parids
    if(canonical_addresses_empty(property_canonical_addr)):
        print "%r: Empty property_canonical_addr, skipping for the future" % (parid)
        ownerprox_skip_parids.add(parid)
        return unknown_color

    # At this point we can trust addrs_match to be True or False and have complete info it needs
    # Check if both addresses are identical use sameaddr_color
    if(addrs_match):
        return sameaddr_color
    
    # Do have both, check for matching zip code
    property_zip=get_canonical_zip(property_canonical_addr)
    owner_zip=get_canonical_zip(owner_canonical_addr)
        
    # Double check that owner_zip is valid
    if(not owner_zip):
        # Nope, return unknown
        return unknown_color
    
    # Owner zip is valid, see if it matches property_zip
    if(property_zip == owner_zip):
        return samezip_color
    
    # Check for in county
    zip_level = 'out-of-state'
    if(owner_zip in zip_map.keys()):
        zip_level = zip_map[owner_zip]
    
    if(zip_level=='Allegheny'):
        return allegheny_co_color
    elif(zip_level=='PA'):
        return in_state_color
    else:
        return out_of_state_color

def output_ownerprox_dots(parid_arr, suffix):
    # If parid_arr not specified, do all of the keys in property_map
    if(len(parid_arr)==0):
        parid_arr=property_map.keys()
        
    # Write out volume of ownership for each residential non-vacant land property
    points = []
    start=arrow.now()
    chunk_start_time=arrow.now()
    chunk_size=1000
    chunk_cnt=0
    
    print "Before removing ownerprox_skip_parids, len(parid_arr) = %d" % (len(parid_arr))
    valid_parid_arr = list(set(parid_arr) - ownerprox_skip_parids)
    print "After removing ownerprox_skip_parids, len(parid_arr) = %d" % (len(valid_parid_arr))
   
    for parid in valid_parid_arr:
        centroid=None
        did_output_dot=False
        try:
            centroid = parid2centroid[parid]
        except:
            print "%s is missing from centroids, removing from future consideration" % (parid)
            ownerprox_skip_parids.add(parid)
            continue

        try:
            # Handle periodic debug message
            if((chunk_cnt%chunk_size)==0 and chunk_cnt>0):
                print "%d-%d: processing %r, %s time elapsed" %(chunk_cnt-(chunk_size-1), chunk_cnt, parid, arrow.now()-chunk_start_time)
                addcnt=0
                chunk_start_time=arrow.now()

            # Get list of property events or this property
            property_events = property_map[parid]

            # Keep track of the date of the last datapoint. 
            last_date = '1900-01-01'
            last_color=0
            
            # Iterate over the property events.  We need to deal with different instances of the same sale date
            # as a batch because there might be multiple color changes before the next sale
            i=0
            while i < len(property_events):
                event_date = property_events[i]['date']
                # Get the set of simultaneous events in order of asofdate
                sevs = get_simultaneous_property_events(property_events,i)
                # Get color for each of these property events
                scolors = map(lambda ev: ownerprox_to_color(parid, ev), sevs)
                
                # Compute the bookends of this set of events.  The start date of the first 
                # color dot will be the date of this sale.  The end date of the last color
                # dot will be the date of the next sale.  We might have color changes in between
                # due to modifications in change adresses without a sale.  The potential
                # transition points are the asofdate of events where the status changes.
                next_start_date = event_date
                next_sell_date = None
                # Check if there's another event after this batch.  If so, set next_sell_date to be the
                # date of the next item
                if(i+len(sevs)<len(property_events)):
                    next_sell_date = property_events[i+len(sevs)]['date']

                if((chunk_cnt%chunk_size)==0):
                    print "  %s (%d): %s = %s [next_sell_date = %s]" % (parid, i, event_date, scolors, next_sell_date)

                last_color = scolors[0]
                output_color = True
                
                for j in range(0, len(sevs)):
                    color = scolors[j]
                    start_date = next_start_date
                    end_date = next_sell_date
                    # If we're not yet at the end, set end_date to the next asofdate
                    if(j<len(sevs)-1):
                        end_date = sevs[j+1]['asofdate']
                        next_start_date = end_date
                    # Calculate epoch time of start and end 
                    start_e = SaledateToEpoch(start_date)
                    if(end_date == None):
                        # No new owner after this, set end date to the end of time
                        end_e = float(1e38)
                    else:
                        end_e = SaledateToEpoch(end_date)

                    if(color != None):
                        points += PointToPixelXY(centroid)     
                        points.append(parse_color(color))
                        # Put epoch time for SALEDATE as start valid time, and next_date as end valid time
                        points.append(float(start_e))
                        points.append(float(end_e))
                        #print "  %s (%d): %s - %s = %s (len %d)" % (parid, i+j, start_date, end_date, color, len(points))
                    else:
                        print "Color of " + str(volume_map['parcount']) + " is None"


                # Set i, last_date and last_color for next loop
                i = i + len(sevs)
                last_date = event_date
                last_color= color
                did_output_dot=True
        except:
            print "Unexpected error processing %s:" % (parid), sys.exc_info()[0]
            raise

        #Increment debug message counter
        if(did_output_dot):
            chunk_cnt = chunk_cnt+1

    array.array('f', points).tofile(open(('assessments/ownerprox_color_m_epoch%s.bin'%suffix), 'wb'))

    end=arrow.now()
    print "Processing took %s for %d dots" % (str(end-start), chunk_cnt)
    

In [ ]:
test_parids=['2018H00310000000','0026N00263000000', '0010H00190000000', '0428D00080000000',
            '0072B00069000000', '0050P00138000000', '0195A00010000000',
            '0233K00148000000', '0033G00182000000', '0067D00114000000',
            '0079A00094000000', '0006G00201000000', '0555S00258000000',
            '0368N00060000000', '0029M00270000000', '1081P00065000000',
            '0130M00122000000', '1238H00385000000', '0381E00028000000',
            '0235J00250000000', '0013S00060000000', '0019M00033000000',
            '0042K00177000000', '0010D00089000000', '1064J00060000000',
            '0596A00245113500', '0739L00354000000', '0738B00123000000',
            '1099P00017000000', '0471D00291000000', '0632S00257000000',
            '0456H00094000000', '0133L00049000000', '0029G00165000000',
            '1223N00336000000', '0087C00130000000', '2018H00310000000']
output_ownerprox_dots(test_parids,"_test_99_04c_05_09_10_17")

In [ ]:
output_ownerprox_dots(valid_parids_15217,"_d1_15217_99b_04d_05_09_10_11_17_18b")

In [ ]:
len(ownerprox_skip_parids)

In [ ]:
# Process the full set of properties
output_ownerprox_dots([],out_suffix)

In [ ]:
2+2

In [ ]:
sorted(property_map['0087D00043000000'], key=lambda ev: ''.join([ev['date'],ev['asofdate']]))

In [ ]:
property_map['0087D00043000000'][0]['date']='05-05-1950'

In [ ]:
owner_map[u'SACKS AARON & SHIRLEY P'][0]['date']='05-05-1950'

In [ ]:
changeaddr_map[u'5925 PHILLIPS AVE PITTSBURGH PA 15217'][0]['date']='05-05-1950'

In [ ]:
apd_1999.loc['0086E00117000000']

In [ ]:
changeaddr_map['2300 E HOLIDAY CT LANSING IL 60438']

In [ ]:
owner_map['HRANTICS STEPHEN A']

In [ ]:
ownerprox_to_color('0553N00123000000', property_map['0553N00123000000'][0])

In [ ]:
raw_property_address_map['0553N00123000000']

In [ ]:
property_map['0232C00201000000'] = [{'asofdate': '1999-01-01',
  'canonical_owner_address': u'6255 LIBRARY RD, BETHEL PARK, PA, 15102',
  'changeaddr': u'6255 LIBRARY RD BETHEL PARK PA 1510',
  'date': '1970-11-01',
  'event_type': 'PURCHASE',
  'ownername': u'SEACH CO INC'}]

changeaddr_map['6255 LIBRARY RD BETHEL PARK PA 1510'] = [{'date': '1970-11-01', 'event_type': 'PURCHASE', 'parid': '0232C00201000000'}]

owner_map['SEACH CO INC'] = [{'date': '1970-11-01', 'event_type': 'PURCHASE', 'parid': '0232C00202000000'},
 {'date': '1970-11-01', 'event_type': 'PURCHASE', 'parid': '0232C00201000000'}]


In [ ]:
output_ownerprox_dots(valid_parids_15213,"_d3_15213")

In [ ]:
output_ownerprox_dots(valid_parids_15213[0:100],"_d3_100_15213")

In [ ]:
# Do new fair market colors based on values stored in property_map

fm_colors = ['#a50026','#cd2827','#e75436','#f7804b','#fdad61','#fed788','#ffffbf','#b9e0ed','#8dc0db','#699fca','#4d7db9','#3e5aa7','#313695']
unknown_color = '#303030'


def fm_total_to_color(fm_total):
    if (fm_total < 25000.):
        return fm_colors[0] 
    elif (fm_total < 50000.):
        return fm_colors[1] 
    elif (fm_total < 75000.):
        return fm_colors[2] 
    elif (fm_total < 100000.):
        return fm_colors[3] 
    elif (fm_total < 125000.):
        return fm_colors[4] 
    elif (fm_total < 150000.):
        return fm_colors[5] 
    elif (fm_total < 200000.):
        return fm_colors[6] 
    elif (fm_total < 250000.):
        return fm_colors[7] 
    elif (fm_total < 300000.):
        return fm_colors[8] 
    elif (fm_total < 400000.):
        return fm_colors[9] 
    elif (fm_total < 500000.):
        return fm_colors[10] 
    elif (fm_total < 750000.):
        return fm_colors[11] 
    else:
        return fm_colors[12] 
 
def output_fairmarket_dots(parid_arr, suffix):
    # If parid_arr not specified, do all of the keys in property_map
    if(len(parid_arr)==0):
        parid_arr=property_map.keys()
        
    # Write out fairmarket value for each residential land property
    # TODO: get rid of vacant?
    points = []
    start=arrow.now()
    chunk_start_time=arrow.now()
    chunk_size=1000
    chunk_cnt=0

    for parid in parid_arr:
        centroid=None
        did_output_dot=False
        try:
            centroid = parid2centroid[parid]
        except:
            print "%s is missing from centroids, skipping" % (parid)
            continue

        try:
            # Handle periodic debug message
            if((chunk_cnt%chunk_size)==0 and chunk_cnt>0):
                print "%d-%d: processing %r, %s time elapsed" %(chunk_cnt-(chunk_size-1), chunk_cnt, parid, arrow.now()-chunk_start_time)
                addcnt=0
                chunk_start_time=arrow.now()

            # Get list of property events or this property
            property_events = property_map[parid]

            # Keep track of the date of the last datapoint. 
            last_date = '1900-01-01'
            last_color=0
            
            # Iterate over the property events.  We need to deal with different instances of the same sale date
            # as a batch because there might be multiple color changes before the next sale
            i=0
            while i < len(property_events):
                event_date = property_events[i]['date']
                # Get the set of simultaneous events in order of asofdate
                sevs = get_simultaneous_property_events(property_events,i)
                # Get color for each of these property events
                scolors = map(lambda ev: fm_total_to_color(ev['fm_total']) if 'fm_total' in ev else unknown_color, sevs)
                
                # Compute the bookends of this set of events.  The start date of the first 
                # color dot will be the date of this sale.  The end date of the last color
                # dot will be the date of the next sale.  We might have color changes in between
                # due to modifications in change adresses without a sale.  The potential
                # transition points are the asofdate of events where the status changes.
                next_start_date = event_date
                next_sell_date = None
                # Check if there's another event after this batch.  If so, set next_sell_date to be the
                # date of the next item
                if(i+len(sevs)<len(property_events)):
                    next_sell_date = property_events[i+len(sevs)]['date']

                if((chunk_cnt%chunk_size)==0):
                    print "  %s (%d): %s = %s [next_sell_date = %s]" % (parid, i, event_date, scolors, next_sell_date)

                last_color = scolors[0]
                output_color = True
                
                for j in range(0, len(sevs)):
                    color = scolors[j]
                    start_date = next_start_date
                    end_date = next_sell_date
                    # If we're not yet at the end, set end_date to the next asofdate
                    if(j<len(sevs)-1):
                        end_date = sevs[j+1]['asofdate']
                        next_start_date = end_date
                    # Calculate epoch time of start and end 
                    start_e = SaledateToEpoch(start_date)
                    if(end_date == None):
                        # No new owner after this, set end date to the end of time
                        end_e = float(1e38)
                    else:
                        end_e = SaledateToEpoch(end_date)

                    # Skip if this is a vacant lot
                    if('is_vacant' in sevs[j] and sevs[j]['is_vacant']):
                        #print "%s: Skipping vacant lot" % (parid)
                        continue

                    if(color != None):
                        points += PointToPixelXY(centroid)     
                        points.append(parse_color(color))
                        # Put epoch time for SALEDATE as start valid time, and next_date as end valid time
                        points.append(float(start_e))
                        points.append(float(end_e))
                        #print "  %s (%d): %s - %s = %s (len %d)" % (parid, i+j, start_date, end_date, color, len(points))
                    else:
                        print "Color of " + str(volume_map['parcount']) + " is None"


                # Set i, last_date and last_color for next loop
                i = i + len(sevs)
                last_date = event_date
                last_color= color
                did_output_dot=True
        except:
            print "Unexpected error processing %s:" % (parid), sys.exc_info()[0]
            #raise

        #Increment debug message counter
        if(did_output_dot):
            chunk_cnt = chunk_cnt+1

    array.array('f', points).tofile(open(('assessments/parcels_fm_color_epoch%s.bin'%suffix), 'wb'))

    end=arrow.now()
    print "Processing took %s for %d dots" % (str(end-start), chunk_cnt)

In [ ]:
output_fairmarket_dots(valid_parids_15213,"_d2_15213")

In [ ]:
# Process the full set of properties
output_fairmarket_dots([],out_suffix)

In [ ]:
property_map['0086K00135000000']

In [ ]:
property_map['0174E00234000000']

In [ ]:
def vacancy_to_color(pev):
    # default to grey
    color = "#333333"
    if('is_vacant' in pev and pev['is_vacant']):
        # vacant lot, make it green
        color = "#02ca75"
    return color
 
def output_vacancy_dots(parid_arr, suffix):
    # If parid_arr not specified, do all of the keys in property_map
    if(len(parid_arr)==0):
        parid_arr=property_map.keys()
        
    # Write out whether or not land is vacant for each parcel
    points = []
    start=arrow.now()
    chunk_start_time=arrow.now()
    chunk_size=1000
    chunk_cnt=0

    for parid in parid_arr:
        centroid=None
        did_output_dot=False
        try:
            centroid = parid2centroid[parid]
        except:
            print "%s is missing from centroids, skipping" % (parid)
            continue

        try:
            # Handle periodic debug message
            if((chunk_cnt%chunk_size)==0 and chunk_cnt>0):
                print "%d-%d: processing %r, %s time elapsed" %(chunk_cnt-(chunk_size-1), chunk_cnt, parid, arrow.now()-chunk_start_time)
                addcnt=0
                chunk_start_time=arrow.now()

            # Get list of property events or this property
            property_events = property_map[parid]

            # Keep track of the date of the last datapoint. 
            last_date = '1900-01-01'
            last_color=0
            
            # Iterate over the property events.  We need to deal with different instances of the same sale date
            # as a batch because there might be multiple color changes before the next sale
            i=0
            while i < len(property_events):
                event_date = property_events[i]['date']
                # Get the set of simultaneous events in order of asofdate
                sevs = get_simultaneous_property_events(property_events,i)
                # Get color for each of these property events
                scolors = map(lambda ev: vacancy_to_color(ev), sevs)
                
                # Compute the bookends of this set of events.  The start date of the first 
                # color dot will be the date of this sale.  The end date of the last color
                # dot will be the date of the next sale.  We might have color changes in between
                # due to modifications in change adresses without a sale.  The potential
                # transition points are the asofdate of events where the status changes.
                next_start_date = event_date
                next_sell_date = None
                # Check if there's another event after this batch.  If so, set next_sell_date to be the
                # date of the next item
                if(i+len(sevs)<len(property_events)):
                    next_sell_date = property_events[i+len(sevs)]['date']

                if((chunk_cnt%chunk_size)==0):
                    print "  %s (%d): %s = %s [next_sell_date = %s]" % (parid, i, event_date, scolors, next_sell_date)

                last_color = scolors[0]
                output_color = True
                
                for j in range(0, len(sevs)):
                    color = scolors[j]
                    start_date = next_start_date
                    end_date = next_sell_date
                    # If we're not yet at the end, set end_date to the next asofdate
                    if(j<len(sevs)-1):
                        end_date = sevs[j+1]['asofdate']
                        next_start_date = end_date
                    # Calculate epoch time of start and end 
                    start_e = SaledateToEpoch(start_date)
                    if(end_date == None):
                        # No new owner after this, set end date to the end of time
                        end_e = float(1e38)
                    else:
                        end_e = SaledateToEpoch(end_date)

                    if(color != None):
                        points += PointToPixelXY(centroid)     
                        points.append(parse_color(color))
                        # Put epoch time for SALEDATE as start valid time, and next_date as end valid time
                        points.append(float(start_e))
                        points.append(float(end_e))
                        #print "  %s (%d): %s - %s = %s (len %d)" % (parid, i+j, start_date, end_date, color, len(points))
                    else:
                        print "Color of " + str(volume_map['parcount']) + " is None"


                # Set i, last_date and last_color for next loop
                i = i + len(sevs)
                last_date = event_date
                last_color= color
                did_output_dot=True
        except:
            print "Unexpected error processing %s:" % (parid), sys.exc_info()[0]
            #raise

        #Increment debug message counter
        if(did_output_dot):
            chunk_cnt = chunk_cnt+1

    array.array('f', points).tofile(open(('assessments/vacant_lots_m_epoch%s.bin'%suffix), 'wb'))

    end=arrow.now()
    print "Processing took %s for %d dots" % (str(end-start), chunk_cnt)

In [ ]:
output_vacancy_dots(valid_parids_15213,"_d1_15213")

In [ ]:
# Process the full set of properties
output_vacancy_dots([],out_suffix)

# Graveyard

In [ ]:
# Below is Anne's attempt to put floating point representations of colors into the .bin file for a new 
# shader Randy's writing

In [ ]:
def incrementKey(adict,key):
    if(key == '' or math.isnan(key)):
       return
    if not adict.has_key(key):
        adict[key] = 1
    else:
        adict[key] += 1

In [ ]:
year_built_colors = ['#ffffe0','#ffdaa3','#ffb27c','#fb8768','#eb5f5b','#d3394a','#b3152f','#8b0000']
def year_built_to_color(year):
    if (year <= 1900.0):
        return color_from_floats(64,64,64)
    elif (year < 1916.0):
        return parse_color(year_built_colors[7])       
    elif (year < 1932.0):
        return parse_color(year_built_colors[6])
    elif (year < 1948.0):
        return parse_color(year_built_colors[5])
    elif (year < 1964.0):
        return parse_color(year_built_colors[4])
    elif (year < 1980.0):
        return parse_color(year_built_colors[3])
    elif (year < 1996.0):
        return parse_color(year_built_colors[2])
    elif  (year < 2012.0):
        return parse_color(year_built_colors[1])
    elif  (year < 2016.0):
        return parse_color(year_built_colors[0])
    else:
        return color_from_floats(255,255,255)
 
   
points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    year_built = apd['YEARBLT'][i]
    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(year_built_to_color(float(year_built)))
        # Set start valid time as year_built, and max positive float as end valid time
        date = datetime.datetime.strptime(year_built, '%Y')        
        points.append(GetEpoch(date))
        points.append(float(1e38))
array.array('f', points).tofile(open(('assessments/parcels_bltyr_color_epoch%s.bin'%out_suffix), 'wb'))

In [ ]:
# Hightlight based on likelyhood of lead
# <1950 = Probably lead
# 1950-1978 = Maybe lead (not illegal, but being phased out)
# >1978 = Probably not lead (illegal)
year_built_colors = ['#ffffe0','#ffdaa3','#ffb27c','#fb8768','#eb5f5b','#d3394a','#b3152f','#8b0000']
def year_built_to_lead_color(year):
    if (year <= 1900.0):
        return color_from_floats(64,64,64)
    elif (year < 1950.0):
        return parse_color(year_built_colors[7])       
    elif (year < 1979.0):
        return parse_color(year_built_colors[3])
    else:
        return parse_color(year_built_colors[0])
    
points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    year_built = apd['YEARBLT'][i]
    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(year_built_to_lead_color(float(year_built)))
        # Set start valid time as year_built, and max positive float as end valid time
        date = datetime.datetime.strptime(year_built, '%Y')        
        points.append(GetEpoch(date))
        points.append(float(1e38))
array.array('f', points).tofile(open(('assessments/parcels_bltyr_lead_color_epoch%s.bin'%out_suffix), 'wb'))

In [ ]:
parcel_colors = ['#fb3059','#fe6b2d','#d1947a','#c6a900','#02ca75','#00a2de','#9529b1']
class_descriptions = sorted(class_descs.keys())

def class_desc_to_color(class_description):
    index = class_descriptions.index(class_description)
    return(parse_color(parcel_colors[index]))

points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    class_description = apd['CLASSDESC'][i] 
    year_built = apd['YEARBLT'][i]

    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)

    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(class_desc_to_color(class_description))
        # Set start valid time as year_built, and max positive float as end valid time
        date = datetime.datetime.strptime(year_built, '%Y')        
        points.append(GetEpoch(date))
        points.append(float(1e38))
array.array('f', points).tofile(open(('assessments/parcels_class_color_epoch%s.bin'%out_suffix), 'wb'))

In [ ]:
# Highlight vacant lots
points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    class_description = apd['CLASSDESC'][i] 
    usedesc = apd['USEDESC'][i]

    # default to grey
    color = "#555555"
    if(usedesc =='VACANT LAND'):
        # vacant lot, make it green
        color = "#02ca75"

    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(parse_color(color))
        # Set start valid time as forever, and max positive float as end valid time
        points.append(0)
        points.append(float(1e38))
array.array('f', points).tofile(open(('assessments/vacant_lots%s.bin'%out_suffix), 'wb'))

In [ ]:
# Here are Anne's efforts using the version with owner names to generate volume of properties owned by current owner

In [ ]:
# SALEDATE field is in '%m-%d-%Y' format (ex 10-26-2012) when present
def SaledateToEpoch(datestr):
    return calendar.timegm(time.strptime(datestr, '%m-%d-%Y'))

In [ ]:
# Parse apd table into dictionaries from owner names and change addresses to parid, and from parid to other info
# about the property
owner_names_to_parids = {}
owner_changeaddrs_to_parids = {}
parid_info = {}
volume_counts = {}

# For now, only look at residential properties with known centroids in 15213
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    zipcode = apd['PROPERTYZIP'][i]
    class_description = apd['CLASSDESC'][i] 
    usedesc = apd['USEDESC'][i] 
    if (usedesc == 'VACANT LAND' or not centroids.has_key(par_id)):
        # skip this one
        continue
    
    # We want to include residential, apartments (usedesc includes APART)
    if(class_description != 'RESIDENTIAL' and (isinstance(usedesc, numbers.Number) or 'APART' not in usedesc)):
        # skip this one
        continue
        
    # This one meets our criteria
    owner_name = apd['PROPERTYOWNER'][i]
    owner_changeaddr = string.strip(str(apd['CHANGENOTICEADDRESS1'][i]))
    c2 = string.strip(str(apd['CHANGENOTICEADDRESS2'][i]))
    c3 = string.strip(str(apd['CHANGENOTICEADDRESS3'][i]))
    c4 = apd['CHANGENOTICEADDRESS4'][i]
    if(c2 != ''):
        owner_changeaddr+=", " + c2
    if(c3 != ''):
        owner_changeaddr+=", " + c3
    if(c4 != ''):
        if(isinstance(c4, numbers.Number)):
            if(not math.isnan(float(c4))):
                owner_changeaddr+=", " + str(int(c4))
        else:
            owner_changeaddr+=", " + c4
        
    if(owner_changeaddr == ''):
        # skip this one
        continue
        
    if(owner_names_to_parids.has_key(owner_name)):
        owner_names_to_parids[owner_name].append(par_id)
    else:
        owner_names_to_parids[owner_name]=[par_id]
        
    if(owner_changeaddrs_to_parids.has_key(owner_changeaddr)):
        owner_changeaddrs_to_parids[owner_changeaddr].append(par_id)
    else:
        owner_changeaddrs_to_parids[owner_changeaddr]=[par_id]

    parid_info[par_id]={'owner_name':owner_name,
                        'owner_changeaddr':owner_changeaddr,
                        'centroid':centroids[par_id],
                        'HOMESTEADFLAG':apd['HOMESTEADFLAG'][i],
                        'USEDESC':apd['USEDESC'][i],
                        'OWNERDESC':apd['OWNERDESC'][i],
                        'SALEDATE':apd['SALEDATE'][i],
                        'OWNERDESC':apd['OWNERDESC'][i],
                        'CLASSDESC':apd['CLASSDESC'][i]
                       }

In [ ]:
# Add counts to parid_info based on max of owner_names_to_parids or owner_changeaddrs_to_parids
# Special case properties with HOMESTEADFLAG set to HOM to be volume=1
for par_id in parid_info.keys():
    volume=1
    if(parid_info[par_id]['HOMESTEADFLAG']!='HOM'):
        ownername_num = len(owner_names_to_parids[parid_info[par_id]['owner_name']])
        ownerchangeaddr_num = len(owner_changeaddrs_to_parids[parid_info[par_id]['owner_changeaddr']])
        volume = max(ownername_num,ownerchangeaddr_num)
    parid_info[par_id]['volume'] = volume
    incrementKey(volume_counts,volume)

In [ ]:
volume_counts

In [ ]:
for ownername in owner_names_to_parids.keys():
    ownername_num = len(owner_names_to_parids[ownername])
    if(ownername_num>10):
        print "%s: %d" % (ownername,ownername_num)

In [ ]:
for ownerchangeaddr in owner_changeaddrs_to_parids.keys():
    ownerchangeaddr_num = len(owner_changeaddrs_to_parids[ownerchangeaddr])
    if(ownerchangeaddr_num>10):
        print "%s: %d" % (ownerchangeaddr,ownerchangeaddr_num)

In [ ]:
for par_id in parid_info.keys():
    if(parid_info[par_id]['volume']>10):
        print parid_info[par_id]

In [ ]:
# Write out a binary file with the volume colors
vol_colors = ['#a50026','#cd2827','#e75436','#f7804b','#fdad61','#fed788','#ffffbf','#b9e0ed','#8dc0db','#699fca','#4d7db9','#3e5aa7','#313695']
def volume_to_color(volume):
    if (volume < 2):
        return parse_color(vol_colors[0]) 
    #elif (volume < 3):
    #    return parse_color(vol_colors[1]) 
    #elif (volume < 4):
    #    return parse_color(vol_colors[2]) 
    elif (volume < 5):
        return parse_color(vol_colors[3]) 
    elif (volume < 10):
        return parse_color(vol_colors[4]) 
    elif (volume < 20):
        return parse_color(vol_colors[5]) 
    elif (volume < 40):
        return parse_color(vol_colors[6]) 
    elif (volume < 60):
        return parse_color(vol_colors[7]) 
    elif (volume < 80):
        return parse_color(vol_colors[8]) 
    elif (volume < 150):
        return parse_color(vol_colors[9]) 
    elif (volume < 300):
        return parse_color(vol_colors[10]) 
    elif (volume < 500):
        return parse_color(vol_colors[11]) 
    else:
        return parse_color(vol_colors[12]) 

# Write out volume of ownership for each residential non-vacant land property
points = []
for par_id in parid_info.keys():
    centroid = parid_info[par_id]['centroid']
    color = volume_to_color(parid_info[par_id]['volume'])
    saledate = SaledateToEpoch('01-01-1900')
    saledate_raw = parid_info[par_id]['SALEDATE']
    if(not(saledate_raw == '' or (isinstance(saledate_raw, numbers.Number) and math.isnan(saledate_raw)))):
        # If not valid, leave as 1900, otherwise parse it into an epoch time
        saledate = SaledateToEpoch(saledate_raw)
        
    if(color != None):
        points += LonLatToPixelXY(centroid)        
        points.append(color)
        # Put epoch time for SALEDATE as start valid time, and max positive float as end valid time
        points.append(float(saledate))
        points.append(float(1e38))
    else:
        print "Color of " + str(parid_info[par_id]['volume']) + " is None"
array.array('f', points).tofile(open(('assessments/res_volume_color_epoch%s.bin'%out_suffix), 'wb'))

In [ ]:
#original version
legend = '<svg width="400" height="30"><rect fill="#a50026" x="0" height="10" width="30"></rect><rect fill="#cd2827" x="30" height="10" width="30"></rect><rect fill="#e75436" x="60" height="10" width="30"></rect><rect fill="#f7804b" x="90" height="10" width="30"></rect><rect fill="#fdad61" x="120" height="10" width="30"></rect><rect fill="#fed788" x="150" height="10" width="30"></rect><rect fill="#ffffbf" x="180" height="10" width="30"></rect><rect fill="#b9e0ed" x="210" height="10" width="30"></rect><rect fill="#8dc0db" x="240" height="10" width="30"></rect><rect fill="#699fca" x="270" height="10" width="30"></rect><rect fill="#4d7db9" x="300" height="10" width="30"></rect><rect fill="#3e5aa7" x="330" height="10" width="30"></rect><text font-size="10.5px" y="29" x="0" fill="#ffffff">1</text><text font-size="10.5px" y="29" x="20"  fill="#ffffff">&nbsp;2</text><text font-size="10.5px" y="29" x="50"  fill="#ffffff">&nbsp;3</text><text font-size="10.5px" y="29" x="80"  fill="#ffffff"> 4</text><text font-size="10.5px" y="29" x="110"  fill="#ffffff"> 5</text><text font-size="10.5px" y="29" x="140"  fill="#ffffff">10</text><text font-size="10.5px" y="29" x="170"  fill="#ffffff">20</text><text font-size="10.5px" y="29" x="200"  fill="#ffffff">40</text><text font-size="10.5px" y="29" x="230"  fill="#ffffff">60</text><text font-size="10.5px" y="29" x="260"  fill="#ffffff">80</text><text font-size="10.5px" y="29" x="290"  fill="#ffffff">150</text><text font-size="10.5px" y="29" x="320"  fill="#ffffff">300</text><text font-size="10.5px" y="29" x="350"  fill="#ffffff">500</text>'
HTML(legend)

In [ ]:
#modified version
legend = '<svg width="400" height="30"><rect fill="#a50026" x="0" height="10" width="30"></rect><rect fill="#f7804b" x="30"  height="10" width="30"></rect><rect fill="#fdad61" x="60"  height="10" width="30"></rect><rect fill="#fed788" x="90"  height="10" width="30"></rect><rect fill="#ffffbf" x="120" height="10" width="30"></rect><rect fill="#b9e0ed" x="150" height="10" width="30"></rect><rect fill="#8dc0db" x="180" height="10" width="30"></rect><rect fill="#699fca" x="210" height="10" width="30"></rect><rect fill="#4d7db9" x="240" height="10" width="30"></rect><rect fill="#3e5aa7" x="270" height="10" width="30"></rect><text font-size="10.5px" y="29" x="0" fill="#ffffff">1</text><text font-size="10.5px" y="29" x="20"  fill="#ffffff">&nbsp;2</text><text Font-size="10.5px" y="29" x="50"   fill="#ffffff"> 5</text><text font-size="10.5px" y="29" x="80"   fill="#ffffff">10</text><text font-size="10.5px" y="29" x="110"  fill="#ffffff">20</text><text font-size="10.5px" y="29" x="140"  fill="#ffffff">40</text><text font-size="10.5px" y="29" x="170"  fill="#ffffff">60</text><text font-size="10.5px" y="29" x="200"  fill="#ffffff">80</text><text font-size="10.5px" y="29" x="230"  fill="#ffffff">150</text><text font-size="10.5px" y="29" x="260"  fill="#ffffff">300</text><text font-size="10.5px" y="29" x="290"  fill="#ffffff">500</text>'
HTML(legend)

In [ ]:
# Write out a binary file with the colors for residential owned by corporate, government, or regular owners
def ownertype_to_color(par_id):
    ownerdesc = parid_info[par_id]['OWNERDESC']
    classdesc = parid_info[par_id]['CLASSDESC']
    volume = parid_info[par_id]['volume']
    if (classdesc == "GOVERNMENT"):
        print "Government: %s, %s, %d" % (ownerdesc, classdesc,volume)
        # Same color as government in property class view
        return parse_color("#d1947a") 
    elif ('CORPORATION' in ownerdesc):
        # Same color as 500+ properties in volume view
        #return parse_color('#313695') 
        return parse_color('#ffffbf')
    elif ('REGULAR' in ownerdesc):
        # Regular owner, what volume?
        if(volume == 1):
            # Same color as 1 in volume view
            return parse_color('#a50026')
        else:
            # Same color as other in class view
            return parse_color('#02ca75')
    else:
        print "Unrecognized owner type: %s, %s, %d" % (ownerdesc, classdesc,volume)
        return parse_color('#303030') 

# Write out volume of ownership for each residential non-vacant land property
points = []
for par_id in parid_info.keys():
    centroid = parid_info[par_id]['centroid']
    color = ownertype_to_color(par_id)
    saledate = SaledateToEpoch('01-01-1900')
    saledate_raw = parid_info[par_id]['SALEDATE']
    if(not(saledate_raw == '' or (isinstance(saledate_raw, numbers.Number) and math.isnan(saledate_raw)))):
        # If not valid, leave as 1900, otherwise parse it into an epoch time
        saledate = SaledateToEpoch(saledate_raw)
        
    if(color != None):
        points += LonLatToPixelXY(centroid)        
        points.append(color)
        # Put epoch time for SALEDATE as start valid time, and max positive float as end valid time
        points.append(float(saledate))
        points.append(float(1e38))
    else:
        print "Color of " + str(parid_info[par_id]['volume']) + " is None"
array.array('f', points).tofile(open(('assessments/ownertype_color_epoch%s.bin'%out_suffix), 'wb'))

In [ ]:
parid_info['0002K00028000000']

In [ ]:
# Foreclosures

In [ ]:
# SALEDATE field is in '%m-%d-%Y' format (ex 10-26-2012) when present
def FilingdateToEpoch(datestr):
    return calendar.timegm(time.strptime(datestr, '%Y-%m-%d'))

In [ ]:
# Read in foreclosure data 
fpath = "assessments/foreclosures-180327.csv"
fapd = pandas.read_csv(fpath)

In [ ]:
# Write out volume of ownership for each residential non-vacant land property
points = []
missing_pin_c=0
found_pin_c=0
missing_pin_a=0
found_pin_a=0
foreclosure_color = '#ff00ff'

for i in range(0, len(fapd.index)):
    par_id = fapd['pin'][i]
    filing_date_raw = fapd['filing_date'][i]
    
    if(filing_date_raw == '' or (isinstance(filing_date_raw, numbers.Number) and math.isnan(filing_date_raw))):
        print "%d: Can't read filing_date %s" % (i, filing_date_raw)
        continue
    filing_date = FilingdateToEpoch(filing_date_raw)
    
    if par_id not in parid_info:
        missing_pin_c+=1
        continue
    found_pin_c+= 1
    
    centroid = parid_info[par_id]['centroid']
    
    if par_id not in parid_info:
        missing_pin_a+=1
    found_pin_a+=1
    
    saledate = SaledateToEpoch('01-01-1900')
    saledate_raw = parid_info[par_id]['SALEDATE']
    if(not(saledate_raw == '' or (isinstance(saledate_raw, numbers.Number) and math.isnan(saledate_raw)))):
        # If not valid, leave as 1900, otherwise parse it into an epoch time
        saledate = SaledateToEpoch(saledate_raw)
        
    startdate = filing_date
    enddate = saledate
    
    if(filing_date<saledate):
        # Sold after foreclosure, who bought it?
        points += LonLatToPixelXY(centroid)        
        points.append(ownertype_to_color(par_id))
        # Put epoch time for SALEDATE as start valid time, and max positive float as end valid time
        points.append(float(filing_date))
        points.append(float(1e38))
    else:
        # Sold before foreclosure, who owned it before?
        points += LonLatToPixelXY(centroid)        
        points.append(ownertype_to_color(par_id))
        # Put epoch time for SALEDATE as start valid time, and max positive float as end valid time
        points.append(float(saledate))
        points.append(float(filing_date))
        # Not sold after foreclosure, show ownership before sale and end foreclosure at end of time
        enddate=float(1e38)
        
    # Output foreclosure dot
    points += LonLatToPixelXY(centroid)        
    points.append(parse_color(foreclosure_color))
    # Put epoch time for SALEDATE as start valid time, and max positive float as end valid time
    points.append(float(startdate))
    points.append(float(enddate))

array.array('f', points).tofile(open(('assessments/foreclosure_color_epoch%s.bin'%out_suffix), 'wb'))

In [ ]:
# Below here are modified versions of the blocks from Gabriel's original python notebook,
# none of which are needed anymore

In [ ]:
sale_prices = {}
for i in range(0, len(apd.index)):
    sale_price = apd['SALEPRICE'][i]
    if not sale_prices.has_key(sale_price):
        sale_prices[sale_price] = 1
    else:
        sale_prices[sale_price] += 1


In [ ]:
len(sale_prices.keys())


In [ ]:
years_built = {}
sale_dates = {}
fairmarket_totals = {}
classes = {}
class_descs = {}

def incrementKey(adict,key):
    if(key == '' or math.isnan(key)):
       return
    if not adict.has_key(key):
        adict[key] = 1
    else:
        adict[key] += 1
    
for i in range(0, len(apd.index)):
    year_built = apd['YEARBLT'][i]
    incrementKey(years_built, year_built)
    sale_date = apd['SALEDATE'][i]
    incrementKey(sale_dates, sale_date)
    fairmarket_total = apd['FAIRMARKETTOTAL'][i]
    incrementKey(fairmarket_totals, fairmarket_total)
    class_ = apd['CLASS'][i]
    incrementKey(classes, class_)
    class_desc = apd['CLASSDESC'][i]
    incrementKey(class_descs, class_desc)


In [ ]:
i

In [ ]:
len(apd.index)

In [ ]:
len(years_built.keys())
years_built['']

In [ ]:
years_built

In [ ]:
points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    year_built = apd['YEARBLT'][i]
    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)
        date = datetime.datetime.strptime(year_built, '%Y')        
        points.append(GetEpoch(date))
array.array('f', points).tofile(open(('assessments/parcels_yr%s.bin'%out_suffix), 'wb'))


In [ ]:
year_built
date = datetime.datetime.strptime('2000', '%Y')   
GetEpoch(date)

In [ ]:
fairmarket_totals['0']

In [ ]:
classes

In [ ]:
class_descs

In [ ]:
sorted(class_descs.keys())

In [ ]:
no_year_built = 0
no_= 0
for i in range(0, len(apd.index)):
    year_built = apd['YEARBLT'][i]
    class_ = apd['CLASS'][i]
    if year_built == '' and class_ == 'R':
        no_year_built += 1
    elif year_built == '':
        no_ += 1
print no_year_built
print no_

In [ ]:
class_descriptions = sorted(class_descs.keys())
points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    year_built = apd['YEARBLT'][i]
    class_description = apd['CLASSDESC'][i]  
    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)
        date = datetime.datetime.strptime(year_built, '%Y')        
        points.append(GetEpoch(date))
        points.append(class_descriptions.index(class_description))
array.array('f', points).tofile(open(('assessments/parcels_yrblt%s.bin'%out_suffix), 'wb'))


In [ ]:
foo = sorted(class_descs.keys())

In [ ]:
len(foo)

In [ ]:
parcel_colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628']

In [ ]:
for color in hex_colors:
    rgb_color = HexToRgb(color)
    print "%s,%s,%s" % (rgb_color[0]/255.0, rgb_color[1]/255.0, rgb_color[2]/255.0) 

In [ ]:
hex_colors = ["#fb3059",
"#fe6b2d",
"#d1947a",
"#c6a900",
"#02ca75",
"#00a2de",
"#9529b1"]
for color in hex_colors:
    rgb_color = HexToRgb(color)
    print "%.4f,%.4f,%.4f" % (rgb_color[0]/255.0, rgb_color[1]/255.0, rgb_color[2]/255.0) 

In [ ]:
hex_colors = ['#ffffff','#fffffa','#fffdf5','#fffdf1','#fffcef','#fffbed','#fffbea','#fff9e6','#fff9e3','#fff7e0','#fff7dd','#fff6db','#fff5d8','#fff4d6','#fff3d4','#fff3d2','#fff2d0','#fff0cd','#fff0cb','#ffefc8','#ffeec6','#ffedc3','#ffecc1','#ffeabe','#ffeabc','#ffe9b9','#ffe7b7','#ffe7b4','#ffe6b2','#ffe4b0','#ffe4ad','#ffe3ab','#ffe1a8','#ffe1a6','#ffe0a4','#ffdea1','#ffde9f','#ffdc9d','#ffdc9b','#ffda98','#ffd996','#ffd994','#ffd891','#ffd68f','#ffd68d','#ffd58a','#ffd388','#ffd386','#ffd284','#ffd081','#ffcf7f','#ffcf7d','#ffcd7b','#ffcc78','#ffcc76','#ffca74','#ffc971','#ffc96f','#ffc76d','#ffc76b','#ffc568','#ffc566','#ffc464','#ffc361','#ffc15d','#ffc05a','#ffbf58','#ffbe56','#ffbc53','#ffbc51','#ffba4e','#ffba4c','#ffb849','#ffb747','#ffb644','#ffb53f','#ffb43d','#ffb23a','#ffb237','#ffb134','#ffaf31','#ffaf2e','#ffad2a','#ffab24','#ffab21','#ffa91c','#ffa818','#ffa813','#ffa60b','#ffa400','#ffa300','#ffa200','#ffa100','#ff9f00','#ff9e00','#ff9d00','#ff9c00','#ff9b00','#ff9900','#ff9800','#ff9700','#ff9500','#ff9500','#ff9300','#ff9100','#ff9000','#ff8f00','#ff8d00','#ff8c00','#ff8a00','#ff8a00','#fe8800','#fe8800','#fe8700','#fe8400','#fd8400','#fd8300','#fd8200','#fd8000','#fc7f00','#fc7d00','#fc7c00','#fc7b00','#fb7a00','#fb7900','#fb7800','#fa7700','#fa7400','#fa7400','#f97200','#f97100','#f87000','#f86e00','#f86e00','#f76d00','#f76c00','#f66a00','#f66900','#f56800','#f56600','#f46600','#f46400','#f36400','#f36200','#f36100','#f26000','#f15e00','#f15d00','#f05d00','#ef5b00','#ef5900','#ef5900','#ee5800','#ed5700','#ed5500','#ec5500','#eb5400','#ea5200','#ea5100','#e95000','#e84f00','#e84e00','#e74c00','#e74b00','#e64a00','#e54900','#e44800','#e44700','#e34600','#e34500','#e14400','#e14300','#e04100','#df4000','#de4000','#de3e00','#dd3d00','#dc3d00','#db3c00','#db3a00','#da3900','#d83800','#d83700','#d73600','#d63500','#d63401','#d43301','#d43201','#d23101','#d13001','#d12e01','#d02e01','#d02d01','#ce2c01','#ce2b01','#cd2a01','#cc2901','#ca2801','#c92701','#c92601','#c82501','#c72401','#c62301','#c52201','#c42102','#c32002','#c21f02','#c01e02','#bf1d02','#bf1c02','#be1b02','#bd1a02','#bc1902','#ba1902','#ba1802','#b81702','#b81602','#b71502','#b51402','#b51302','#b41202','#b21202','#b11102','#b01002','#af0e02','#ae0d02','#ac0d02','#ac0c02','#aa0b02','#a90a02','#a80902','#a80902','#a60802','#a50702','#a30602','#a30602','#a20502','#a00502','#a00402','#9e0302','#9c0302','#9b0302','#9b0201','#9a0201','#990201','#970101','#950101','#950101','#940101','#920001','#920001','#900001','#8f0000','#8d0000','#8d0000','#8b0000']

img = Image.new( 'RGB', (255,255), "white") # create a new image
pixels = img.load() # create the pixel map
for i in range(img.size[0]):    # for every pixel:
    for j in range(img.size[1]):
        pixels[i,j] = hex_to_rgb(hex_colors[i]) # set the colour accordingly
img.show
img.save("assessments/year-built-color-map.png")


In [ ]:
hex_stops =  ['#1d1e4e','#482045','#6f1f4b','#8d2747','#aa3f4c','#a9513f','#ba663a','#cf8139','#e59f44','#e9bb76']
hex_colors = ['#1d1e4e','#201e4e','#241e4d','#261e4d','#291e4c','#291f4c','#2b1f4c','#2d1f4b','#301f4b','#331f4a','#341f4a','#371f49','#381f49','#3b1f48','#3d1f48','#3d1f48','#3f2047','#402047','#432046','#452046','#462046','#482045','#492045','#4b2045','#4d2046','#4d2046','#502046','#502046','#522047','#532047','#552047','#562047','#572047','#5a2048','#5b2048','#5c2048','#5d2048','#5e2048','#602049','#622049','#632049','#652049','#65204a','#67204a','#69204a','#6b1f4a','#6b1f4a','#6c1f4b','#6f1f4b','#701f4b','#711f4b','#72204b','#72204b','#75204a','#75214a','#76214a','#77214a','#78214a','#7a224a','#7a224a','#7d2249','#7d2249','#7e2349','#7f2349','#802449','#812449','#832448','#842548','#852548','#862548','#882548','#882648','#892648','#8b2647','#8b2747','#8d2747','#8e2847','#8e2847','#902947','#902948','#912a48','#922b48','#932c48','#932c48','#952d48','#952d48','#952e49','#972f49','#973049','#983049','#993149','#9a3249','#9a3249','#9c334a','#9c334a','#9d354a','#9d354a','#9f364a','#9f364a','#a0374a','#a1384a','#a2384b','#a2394b','#a33a4b','#a43a4b','#a43b4b','#a63c4b','#a73c4b','#a83d4c','#a83d4c','#a93e4c','#a93f4c','#aa414b','#aa424a','#aa4349','#aa4448','#aa4747','#aa4846','#a94b44','#a94b43','#a94d43','#a94e41','#a94f40','#a9513f','#a9523f','#aa523f','#ab533e','#ac543e','#ac543e','#ad563e','#ad563e','#ae573e','#ae583e','#af593d','#b05a3d','#b05a3d','#b15b3d','#b15b3d','#b25d3c','#b35d3c','#b35e3c','#b45f3c','#b55f3c','#b5603c','#b6623b','#b6623b','#b7623b','#b8643b','#b9643b','#b9653a','#ba653a','#ba673a','#bb683a','#bb683a','#bc683a','#bd693a','#bd6a3a','#bd6a3a','#be6c3a','#bf6c3a','#c06d3a','#c16e3a','#c16e3a','#c16f3a','#c2713a','#c2713a','#c3723a','#c4733a','#c5743a','#c5753a','#c5753a','#c6753a','#c7773a','#c8773a','#c87939','#c97939','#c97939','#ca7a39','#ca7b39','#cb7c39','#cc7d39','#cc7e39','#cd7e39','#ce7f39','#ce8039','#cf8139','#cf8139','#d08239','#d0833a','#d1843a','#d2843a','#d2853b','#d2863b','#d3873b','#d4883b','#d5883c','#d5893c','#d58a3c','#d68b3c','#d78c3d','#d78d3d','#d88d3d','#d88d3e','#d98f3e','#da8f3e','#da903f','#da913f','#db923f','#dc933f','#dc9340','#dd9340','#dd9440','#de9641','#df9641','#df9741','#e09741','#e09942','#e19942','#e19a42','#e29a42','#e29c43','#e39d43','#e49d43','#e59e44','#e59e44','#e59f45','#e5a047','#e6a149','#e6a24b','#e6a44c','#e6a54e','#e6a650','#e6a652','#e6a652','#e7a754','#e7a855','#e7aa57','#e7ab59','#e7ab5b','#e7ac5d','#e8ae5e','#e8af60','#e8af62','#e8b063','#e8b165','#e8b265','#e8b367','#e8b469','#e8b56b','#e9b66c','#e9b76e','#e9b870','#e9b872','#e9ba73','#e9ba75','#e9bb76']

In [ ]:
hex_colors.index(hex_stops[9])

In [ ]:
years_built

In [ ]:
hex_stops = ['#f7fbff','#deebf7','#c6dbef','#9ecae1','#6baed6','#4292c6','#2171b5','#08519c','#08306b']

In [ ]:
for i in hex_stops:
  print i + ","

In [ ]:
hex_stops = ['#ffffe0','#dcfac5','#b8f4ab','#90ee90','#6dc88c','#46a386','#008080','#265a81','#233381','#000080']
for color in hex_stops:
    rgb_color = HexToRgb(color)
    print "%.4f,%.4f,%.4f" % (rgb_color[0]/255.0, rgb_color[1]/255.0, rgb_color[2]/255.0) 

In [ ]:
hex_stops = ['#1d1e4e','#5f2049','#8c2747','#aa414b','#ba653a','#d0823a','#e5a045','#ebc07f','#f7e0af','#ffffe0']
years = [1880,1895,1910,1925,1940,1955,1970,1985,2000,2015]

In [ ]:
for i in range(0,10):
    print "if (year < %s) {" % (years[i])
    rgb_color = HexToRgb(hex_stops[i])
    print "  color = vec4(%.4f,%.4f,%.4f,1.0)" % (rgb_color[0]/255.0, rgb_color[1]/255.0, rgb_color[2]/255.0)
    print "}"

In [ ]:
len(years)

In [ ]:
hex_colors = ['#ffffe0','#ffeec1','#ffdea7','#ffcb91','#ffb880','#ffa474','#fe906a','#f87d64','#f06a5e','#e75758','#db4551','#cf3447','#c0223b','#b0122c','#9e051b','#8b0000']
img = Image.new( 'RGB', (255,255), "white") # create a new image
pixels = img.load() # create the pixel map
for i in range(img.size[0]):    # for every pixel:
    for j in range(img.size[1]):
        pixels[i,j] = hex_to_rgb(hex_colors[i/16]) # set the colour accordingly
img.show()
img.save("assessments/year-built-color-map.png")



In [ ]:
parcel_colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628']
parcel_colors = ['#fb3059','#fe6b2d','#d1947a','#c6a900','#02ca75','#00a2de','#9529b1']

year_built_colors = ['#ffffe0','#ffeec1','#ffdea7','#ffcb91','#ffb880','#ffa474','#fe906a','#f87d64','#f06a5e','#e75758','#db4551','#cf3447','#c0223b','#b0122c','#9e051b','#8b0000']
year_built_colors = ['#ffffe0','#ffdaa3','#ffb27c','#fb8768','#eb5f5b','#d3394a','#b3152f','#8b0000']
img = Image.new( 'RGB', (255,255), "white") # create a new image
pixels = img.load() # create the pixel map
for i in range(0,len(parcel_colors)):
    pixels[0,i] = HexToRgb(parcel_colors[i]) # set the colour accordingly
j = 0
for i in list(reversed(range(len(year_built_colors)))):
    pixels[1,j] = HexToRgb(year_built_colors[i]) # set the colour accordingly
    j += 1
#img.show()
img.save("assessments/color-map.png")


In [ ]:
class_descriptions = sorted(class_descs.keys())
points = []
array.array('f', points).tofile(open('parcels_class.bin', 'wb'))
    par_id = apd['PARID'][i]
    year_built = apd['YEARBLT'][i]
    class_description = apd['CLASSDESC'][i]   
    if year_built == '' or year_built == '0':
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(float(year_built))
        points.append(class_descriptions.index(class_description)+1.0)
array.array('f', points).tofile(open('assessments/parcels_class.bin', 'wb'))


In [ ]:
sorted(class_descs.keys())

In [ ]:
class_descriptions.index('RESIDENTIAL')

In [ ]:
points[10000:10005]

In [ ]:
def scale_year(year):
    if year < 1800.0: 
        year = 1800.0
    return int(((year - 1800.0) * 8.0) / (2016.0-1800.))


In [ ]:
scale_year(1800.0)

In [ ]:
len(year_built_colors)

In [ ]:
j = 0
for i in list(reversed(range(len(year_built_colors)))):
    print '<rect fill="%s" x="%s" height="10" width="25" stroke-width="1px" stroke="#666"></rect>' % (year_built_colors[i],j*25)
    j += 1
    
j = 0
for i in list(reversed(range(len(year_built_colors)))):
    print '<text font-size="10.5px" fill="%s" y="29" x="%s">%s</text>' % (year_built_colors[i], j*25, 1800)
    j += 1


In [ ]:
list(reversed(range(10)))

In [ ]:
len(fairmarket_totals)

In [ ]:
fm_keys = sorted(map(int,fairmarket_totals.keys()))

In [ ]:
numpy.average(fm_keys)

In [ ]:
numpy.mean(fm_keys)

In [ ]:
numpy.std(fm_keys)

In [ ]:
fairmarket_totals['50000']

In [ ]:
parcel_colors = ['#fb3059','#fe6b2d','#d1947a','#c6a900','#02ca75','#00a2de','#9529b1']
year_built_colors = ['#ffffe0','#ffdaa3','#ffb27c','#fb8768','#eb5f5b','#d3394a','#b3152f','#8b0000']
fm_colors = ['#a50026','#cd2827','#e75436','#f7804b','#fdad61','#fed788','#ffffbf','#b9e0ed','#8dc0db','#699fca','#4d7db9','#3e5aa7','#313695'
]

img = Image.new( 'RGB', (255,255), "white") # create a new image
pixels = img.load() # create the pixel map
for i in range(0,len(parcel_colors)):
    pixels[0,i] = HexToRgb(parcel_colors[i]) # set the colour accordingly
j = 0
for i in list(reversed(range(len(year_built_colors)))):
    pixels[1,j] = HexToRgb(year_built_colors[i]) # set the colour accordingly
    j += 1
for i in range(0,len(fm_colors)):
    pixels[2,i] = HexToRgb(fm_colors[i]) # set the colour accordingly
    
#img.show()
img.save("assessments/color-map.png")


In [ ]:
class_descriptions = sorted(class_descs.keys())
points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    year_built = apd['YEARBLT'][i]
    class_description = apd['CLASSDESC'][i] 
    fm_total = float(apd['FAIRMARKETTOTAL'][i])    
    if year_built == '' or year_built == '0' or year_built == '0000' or math.isnan(year_built):
        year_built = '0001' # Null Value, -62135596800.0 Epoch
    elif isinstance(year_built, numbers.Number):
        if(year_built < 1):
            year_built = '0001' # Null Value, -62135596800.0 Epoch
        else:
            year_built = "%04d" % (year_built)
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(float(year_built))
        points.append(class_descriptions.index(class_description)+1.0)
        points.append(fm_total)
array.array('f', points).tofile(open(('assessments/parcels_all%s.bin'%out_suffix), 'wb'))


In [ ]:
sorted(class_descs.keys())

In [ ]:
# Below is Anne's attempt to put floating point representations of colors into the .bin file for a new 
# shader Randy's writing

In [ ]:
parcel_colors = ['#fb3059','#fe6b2d','#d1947a','#c6a900','#02ca75','#00a2de','#9529b1']
class_descriptions = sorted(class_descs.keys())

def class_desc_to_color(class_description):
    index = class_descriptions.index(class_description)
    return(parse_color(parcel_colors[index]))

points = []
for i in range(0, len(apd.index)):
    par_id = apd['PARID'][i]
    class_description = apd['CLASSDESC'][i] 
    if centroids.has_key(par_id):
        centroid = centroids[par_id]
        points += LonLatToPixelXY(centroid)        
        points.append(class_desc_to_color(class_description))
array.array('f', points).tofile(open(('assessments/parcels_class_color%s.bin'%out_suffix), 'wb'))

In [ ]:
len(fm_colors)

In [ ]:
fairmarket_totals['0']

In [ ]:
plt.plot(map(int,fairmarket_totals.keys()), fairmarket_totals.values(), 'ro')
plt.show()

In [ ]:
fm_keys[len(fm_keys) - 100:len(fm_keys)]

In [ ]:
fm_values = ["0", "25K", "50K", "75K", "100K", "125K", "150K", "200K", "250K", "300K", "400K", "500K", "+500K"]
for i in range(len(fm_colors)):
    print '<rect fill="%s" x="%s" height="10" width="30"></rect>' % (fm_colors[i],i*30)
    
for i in range(len(fm_values)):
    print '<text font-size="10.5px" y="29" x="%s">%s</text>' % (i*30, fm_values[i])
